In [ ]:
#| default_exp export

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| include: false
from nbdev import nbdev_export
nbdev_export()

## Imports and templates

In [ ]:
#| exporti

import os
import json
import time
import itertools
import warnings

from collections.abc import Iterable

from copy import deepcopy

import numpy as np
import networkx as nx

from multiprocessing import Pool, Manager, cpu_count
from functools import partial

from redis import ResponseError, Redis

from pygengraph.utils import pathConvert,NpEncoder,adjustZoomLevels
from pygengraph.utils import iset_add
from pygengraph.graph import GenomeGraph
from pygengraph.graph import initialPathAnalysis,calcNodeLengths
from pygengraph.graph import getNodesStructurePathNodeInversionRate,getNextNodePath

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
#| export
# Here are three templates: for component, chunk and main case data structure. It is not a full possible structure, some extra attributes
# can be added during exporting graph.
componentTemplate = {
    "first_bin": 1,
    "last_bin": 1,
    "firstCol": 1,
    "lastCol": 1,
    "occupants": [],
    "matrix": [],
    "larrivals": [],
    "rarrivals": [],
    "ldepartures": [],
    "rdepartures": [],
    "x": 0
}

chunkTemplate = {
    "json_version":19,
    "first_bin": 1,
    "first_col": 1,
    "last_bin": 1,
    "last_col": 1,
    "includes_connectors": True,
    "components": [],
    "compVisCol": {}
}

rootStructTemplate = {
    "json_version": 19,
    "pangenome_length": 0,#pangenomeLength, #total number of bp in pangenome
    "includes_connectors": True,
    "zoom_levels": {}
}

## Functions intro

**Notation and terminology**

In documentation, we refer to graph nucleotides, columns and components. Components contain columns and columns contain nucleotides.

In the code variable names and comments use slightly different notation. Columns in documentation are bins in code and comments, whereas graph nucleotides in documentation are called columns in the code and comments. This happened for the legacy reasons, i.e. originally there was no nucleotide numbers (columns) in the visualised graph structure and components were split into bins (literally, equal sized bins). It is not true anymore, but old terminology left here.

Ideally all variable names and comments should be changes in line with documentation notation, but I have no idea when this can happen.

For various operational or legacy reasons, some of the data structures (usually, lists/array) use 0-based indexing, whereas some others
(usually dicts) can be 0-based or 1-based. Here are the main structures with numerical indexing and their index bases:

- components: keys: 0-based, values: occupants: 0-based, binNumbers: 0-based  
- componentToNode: keys: 0-based, values: 1-based  
- nodeToComponent: keys: 0-based, values: 1-based  
- newToOldInd and oldToNewInd: both index and values are 0-based numbers of components in previous and current zoomlayer.  
- fromLinks: top level keys (from nodes): 1-based, bottom level keys (to nodes): 1-based, values (list of participants): 0-based  
- toLinks: top level keys (to nodes): 1-based, bottom level keys (from nodes): 1-based, values (list of participants): 0-based  
- fromComponentLinks: top level keys (from components): 1-based, bottom level keys (to components): 1-based, values (set of participants): 0-based  
- toComponentLinks: top level keys (to components): 1-based, bottom level keys (from components): 1-based, values (set of participants): 0-based  

## Generating base layer

This set of functions generate the data structures for initial, lowest level zoom (nucleotide or minimum unit resolution). The main orchestration 
function is `baseLayerZoom`.

### Functions

In [ ]:
#| export
def outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,reason,debug=False,inversionThreshold=0.5):
    
    if (nodeInversionInPath<=inversionThreshold):
        rightFarLink = True
    else:
        leftFarLink = True
        
    return leftFarLink,rightFarLink

In [ ]:
#| export
def recordLinks(nodeIdx,nextNode,pathID,step,nodeInversionInPath,nonLinearCond,pathNodeArray,fromLinks,toLinks,debug=False,inversionThreshold=0.5):
    leftFarLink = False
    rightFarLink = False
    if nonLinearCond:
        if debug:
            print(f'Non-linear link for path {pathID} from node {nodeIdx} to node {nextNode}')

        fromLinks.setdefault(nodeIdx,{}).setdefault(nextNode,[]).append(pathID)
        toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
        leftFarLink,rightFarLink = outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,'backward link',debug=debug,inversionThreshold=0.5)
            
    else:
        if np.any([node in pathNodeArray[pathID,:] for node in range(nodeIdx+1*step,nextNode,step)]):
            if debug:
                print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

            fromLinks.setdefault(nodeIdx,{}).setdefault(nextNode,[]).append(pathID)
            toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
            
            leftFarLink,rightFarLink = outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,'forward jumping link',debug=debug,inversionThreshold=0.5)
        else:
            startNode = None
            endNode = None

            if step==-1:
                startNode = nodeIdx
                endNode = nextNode
                if debug:
                    print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')

            elif step==1:
                startNode = nodeIdx
                endNode = nextNode
                if debug:
                    print(f'Intermediate nodes for path {pathID} from node {nodeIdx} to node {nextNode}')
            else:
                if debug:
                    print(f'Jump link for path {pathID} from node {nodeIdx} to node {nextNode}')

                fromLinks.setdefault(nodeIdx,{}).setdefault(nextNode,[]).append(pathID)
                toLinks.setdefault(nextNode,{}).setdefault(nodeIdx,[]).append(pathID)
                leftFarLink,rightFarLink = outLeftRight(nodeInversionInPath,leftFarLink,rightFarLink,'forward jumping link',debug=debug,inversionThreshold=0.5)

            if startNode is not None and endNode is not None and step is not None:
                for intermediateNodeIdx in range(startNode,endNode,step):
                    if debug:
                        print(f'Adding link from node {intermediateNodeIdx} to node {intermediateNodeIdx+1*step} for path {pathID}')
                    fromLinks.setdefault(intermediateNodeIdx,{}).setdefault(intermediateNodeIdx+1*step,[]).append(pathID)
                    toLinks.setdefault(intermediateNodeIdx+1*step,{}).setdefault(intermediateNodeIdx,[]).append(pathID)
    return leftFarLink,rightFarLink,fromLinks,toLinks

In [ ]:
#| export
def checkForBreak(nodeIdx,nodeLen,
                  nodePathsIdx,nodeSeqInPath,
                  uniqueNodePathsIDs,pathNodeCount,
                  pathLengths,
                  pathNodeArray,pathDirArray,
                  occupancy,inversion,
                  fromLinks,toLinks,
                  nBins,maxLengthComponent,
                  blockEdges,
                  inversionThreshold=0.5,
                  debug=False):
    '''
    Function to check whether the component should be broken before (left) and/or after (right) it.
    
    Parameters
    ==========
    
    `nodeIdx`: int. Numerical (1 based) node ID within the graph (consecutive number of the node)
               in the graph order.
    
    #######
    The following 5 parameters are calculated in `nodeStat` function externally and passed here:
    `nodeLen`: int. Length of the node in nucleotides
    `nodePathsIdx`: numpy.array. A 1D numpy.array which contain ids of the paths 
                    for each appearance of the node ID in each path.
    `nodeSeqInPath`: numpy.array. A 1D numpy.array with positions of given node in every path. 
                     The path to which this position is related to can be found from `nodePathsIdx`, 
                     e.g. if `nodeSeqInPath[i]` is `k`, then node with node ID `nodeIdx`
                     appear in path `nodePathsIdx[i]` at position `k`.
    `uniqueNodePathsIDs`: Iterable[int]. Usually a list or 1D numpy.array holding unique IDs of paths 
                          which pass this node at least once.
    `pathNodeCount`: numpy.array.: A 1D array (the same length as `uniqueNodePathsIDs`) which 
                     contains number of times the node appear in each path from `uniqueNodePathsIDs`,
                     e.g. if `pathNodeCount[i]` is `k` then node with ID nodeIdx appears in path
                     `uniqueNodePathsIds[i]` `k` times.
    #######
    
    `occupancy`: dict. A dictionary containing occupancy (number of times given path passed the given 
                 node) of the previous node (in terms of graph order) for each path than passed 
                 through previous node. The dictionary has path IDs as keys.
    `inversion`: dict. A dictionary containing inversion (fraction of copies of the node in the path 
                 that are inverted) of the previous node (in terms of graph order) for each path than 
                 passed through previous node. The dictionary has path IDs as keys.
    `fromLinks`: dict[dict[list]]. Dictionary which holds links from each processed node.
    `toLinks`: dict[dict[list]]. Dictionary which holds links to each processed node.
    `nBins`: int. Number of bins recorded in the current component.
    `maxLengthComponent`: int. Maximum number of bins allowed in a component. Used to break too 
                          long components into smaller more manageable blocks.
    `pathDirArray`: 2D numpy.array. Each row (corresponding to specific path in graph) contains directionality
                    of each node in the path. See `initialPathAnalysis` for details.
    `inversionThreshold`: float (default: 0.5). A threshold after which the node is considered inverted.
    
    Returns
    =======
    
    `leftFarLink`: bool. Shows whether there is a far link on the left that will require component break.
    `rightFarLink`: bool. Shows whether there is a far link on the right that will require component break.
    `leftPathStartEnd`: bool. Shows whether there is a start or end of the path on the left of the node.
    `rightPathStartEnd`: bool. Shows whether there is a start or end of the path on the right of the node.
    `isChangedOccupancy`: bool. Shows whether the occupancy of at least one path significantly changed 
                          from previous node. If previous node was missing, and current node has any 
                          occupancy, it will be False as it does not constitute a component break.
    `isChangeInversion`: bool. Shows whether the inversion of at least one path significantly changed 
                          from previous node.
    `isNotFitIntoComponent`: bool. Shows whether this node won't fit into the currently forming component
                             and it should be broken before the current node.
    `pathStarts`: list. Contains list of path IDs which start from this node.
    `pathEnds`: list. Contains list of path IDs which end on this node.
    `fromLinks`: dict[dict[list]]
    `toLinks`: 
    `occupancy`: dict. A dictionary containing occupancy (number of times given path passed the given 
                 node) of the current node (in terms of graph order) for each path than passed 
                 through previous node. The dictionary has path IDs as keys.
    `inversion`: dict. A dictionary containing inversion (fraction of copies of the node in the path 
                 that are inverted) of the current node (in terms of graph order) for each path than 
                 passed through previous node. The dictionary has path IDs as keys.
    
    '''
    
    # Boolean indicator of far incoming link on the left and right
    leftFarLink = False
    rightFarLink = False

    # Boolean if at least one path starts from this node and start is on the left or right
    leftPathStart = False
    rightPathStart = False
    # Boolean if at least one path ends on this node and end is on the left or right
    leftPathEnd = False
    rightPathEnd = False

    # Indicator whether structure (inversion and occupancy combination over all paths) changed from previous node
    isChangedStructure = nodeIdx-1 in blockEdges
    
    # Indicator whether this node does not fit into current component.
    isNotFitIntoComponent = False

    pathEnds = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if nodeInSeq==pathLengths[pathIdx]-1]))
    pathStarts = np.sort(np.unique([pathIdx for pathIdx,nodeInSeq in zip(nodePathsIdx,nodeSeqInPath) if nodeInSeq==0]))
    
    invertedEnds = set()
    invertedStarts = set()
    
    # Check whether adding this node to the current component will violate the limitation
    # for maximum component length. Is it still needed with proper treatment of partially
    # visible components in the front-end???
    if nBins+nodeLen>maxLengthComponent:
        isNotFitIntoComponent = True
    
    # Check whether component should be broken before current node.
    for j, pathID in enumerate(uniqueNodePathsIDs):
        
        # Calculate occupancy of current node for path
        # It is simply how many times this node is passed by current path.
        occupancy[pathID] = pathNodeCount[j]
        
        # Calculate inversion rate of current node for path
        # It is ratio of how many times the node is passed in reversed direction
        # by the path to overall occupancy of the node by the path.
        nodePositions = np.where(nodePathsIdx==pathID)[0]
        
        nodeInversionInPath = np.sum([int(pathDirArray[pathID,nodeNumInPath]) \
                          for nodeNumInPath in nodeSeqInPath[nodePositions]])\
                          /occupancy[pathID]
        
        # Record current inversion for current pathID for bins generation.
        inversion[pathID] = nodeInversionInPath

        # Get indexes of positions of the node in the path
        
        
        # Check whether the node 
        if nodeInversionInPath<=inversionThreshold:
            # not inverted node
            leftPathStart = leftPathStart or (pathID in pathStarts)
            rightPathEnd = rightPathEnd or (pathID in pathEnds)
        else:
            if debug:
                print(f'Node {nodeIdx}, accession {graph.accessions[pathID]}, inversion {nodeInversionInPath}')
            # inverted node
            rightPathStart = rightPathStart or (pathID in pathStarts)
            
            if pathID in pathStarts:
                invertedStarts.add(pathID)
            
            leftPathEnd = leftPathEnd or (pathID in pathEnds)
            
            if pathID in pathEnds:
                invertedEnds.add(pathID)
        
        for nodePos in nodePositions:
            # Boolean flags to whether check and record incoming and outgoing links
            checkIn = False
            checkOut = False
            # Get the sequencial number of node in the path (for current passing), can be several for duplicated nodes
            nodePositionInPath = nodeSeqInPath[nodePos]
            
            if nodePositionInPath<(pathLengths[pathID]-1):
                # if not last node in the path
                checkOut = True
                nextNode = pathNodeArray[pathID,nodePositionInPath+1]
                nextNodePositions = np.where(pathNodeArray[pathID,:]==nextNode)[0]
                nextNodeInversionInPath = np.sum([int(pathDirArray[pathID,nodeInPathPos]) for nodeInPathPos in nextNodePositions])/len(nextNodePositions)
                if debug:
                    print(f'Node {nodeIdx}, next node {nextNode}, path {pathID}, number in path {nodePositionInPath}, length of path {pathLengths[pathID]}')
            else:
                if debug:
                    print("Last node in path")
            
            if nodePositionInPath>0:
                checkIn = True
                prevNode = pathNodeArray[pathID,nodePositionInPath-1]
                prevNodePositions = np.where(pathNodeArray[pathID,:]==prevNode)[0]
                prevNodeInversionInPath = np.sum([int(pathDirArray[pathID,nodeInPathPos]) for nodeInPathPos in prevNodePositions])/len(prevNodePositions)
                if debug:
                    print(f'Node {nodeIdx}, previous node {nextNode}, path {pathID}, number in path {nodePositionInPath}, length of path {pathLengths[pathID]}')
            else:
                if debug:
                    print("First node in path")
            
            
            if nodeInversionInPath<=inversionThreshold:
            # Not inverted node
            # For the node find the links going to the left of the node
                if checkIn:
                    # node is not the first
                    if debug:
                        print(f"IncomingLinks: {prevNode} -> {nodeIdx}")
                        print(f'Existing known links: {fromLinks.get(prevNode,{})}')

                    if prevNode+1!=nodeIdx:
                        # Link in the path does not go between consecutive nodes (from previous to current)
                        # This means that there is a links coming to the left of the node
                        if (prevNode<nodeIdx and nodeIdx in fromLinks.get(prevNode,{})) or (prevNode>=nodeIdx):
                            # If we are not picking up passing through empty blocks as far link
                            # Tandem duplicates are included as they cause an arrow to the left of the node.
                            leftFarLink = True
                if checkOut:
                    # node is not the last
                    
                    # Find out whether it is a non-linear (e.g. reversed link or link from inversed to non-inversed node).
                    if nextNodeInversionInPath<=inversionThreshold:
                        # The inversion is the same for two consecutive (in path) nodes
                        # Link will be non-linear if previous node is staying further in linearised graph
                        nonLinearCond = (nodeIdx>=nextNode)
                        # Step for checking for jumping over link
                        step = 1
                    else:
                        # The inversion is not the same for two consecutive nodes.
                        nonLinearCond = True
                        # Step is not used and assigned for consistency
                        step =0
                        
                    # Process links from (!) this node.
                    _leftFarLink,_rightFarLink,fromLinks,toLinks = \
                    recordLinks(nodeIdx,nextNode,pathID,
                                step,nodeInversionInPath,
                                nonLinearCond,
                                pathNodeArray,
                                fromLinks,toLinks,
                                debug=debug,
                                inversionThreshold=inversionThreshold)
                    
                    leftFarLink = leftFarLink or _leftFarLink
                    rightFarLink = rightFarLink or _rightFarLink
                            
            else:
                #Inverted node
                # For the node find the links goint from the left of the node
                if checkOut:
                    # node is not the last
                    # Find out whether it is a non-linear (e.g. reversed link or link from inversed to non-inversed node).
                    if nextNodeInversionInPath>inversionThreshold:
                        # The inversion is the same for two consecutive (in path) nodes

                        # Link will be non-linear if previous node is staying further in linearised graph
                        nonLinearCond  = (nodeIdx<=nextNode)
                        # Step for checking for jumping over link
                        step = -1
                    else:
                        # The inversion is not the same for two consecutive nodes.
                        nonLinearCond = True
                        # Step is not used and assigned for consistency
                        step = 0
                        
                    _leftFarLink,_rightFarLink,fromLinks,toLinks = \
                    recordLinks(nodeIdx,nextNode,pathID,
                                step,nodeInversionInPath,
                                nonLinearCond,
                                pathNodeArray,
                                fromLinks,toLinks,
                                debug=debug,
                                inversionThreshold=inversionThreshold)
                    
                    leftFarLink = leftFarLink or _leftFarLink
                    rightFarLink = rightFarLink or _rightFarLink
                
                if checkIn:
                    # The node is not the first
                    if debug:
                        print(f"IncomingLinks: {nodeIdx} -> {nextNode}")
                        print(f'Existing known links: {toLinks.get(nextNode,{})}')

                    if nodeIdx-1!=nextNode:
                        # Link does not go between consecutive blocks (from this to previous one)
                        if (nodeIdx>nextNode and nodeIdx in toLinks.get(nextNode,{})) or (nodeIdx<=nextNode):
                            # If we are not picking up passing through empty blocks as far link
                            # Tandems duplicates are included as they also cause an arrow from the left of the node.
                            rightFarLink = True
        
    return (leftFarLink,rightFarLink,
            leftPathStart or leftPathEnd,
            rightPathStart or rightPathEnd,
            isChangedStructure,
            isNotFitIntoComponent,
            pathStarts,pathEnds,
            fromLinks,toLinks,
            occupancy,inversion,
            invertedStarts,invertedEnds)

In [ ]:
#| export
def nodeStat(nodeIdx,pathNodeArray,nodeLengths):
    '''
    Function calculate information about node as part of the overall graph.
    
    Parameters
    ==========
    
    `nodeIdx`: int. Number of the node in the graph (1 based).
    `pathNodeArray`: numpy.array. A 2D array (<number of paths> x <max lengths of path>), 
                     where each line present sequence of node IDs in a particular path.
                     Path ordered as they are present in the graph.
    `nodeLengths`: list or numpy.array. An 1D subscribable structure 
                   (normally, list or 1D numpy.array is expected) where each element i
                   is the length of the node ID (1 based) i+1.
    
    Return
    ======
    
    `nodePathsIdx`: numpy.array. A 1D numpy.array which contain ids of the paths 
                    for each appearance of the node ID in each path.
    `nodeSeqInPath`: numpy.array. A 1D numpy.array with positions of given node in every path. 
                     The path to which this position is related to can be found from `nodePathsIdx`, 
                     e.g. if `nodeSeqInPath[i]` is `k`, then node with node ID `nodeIdx`
                     appear in path `nodePathsIdx[i]` at position `k`.
    `nodeLen`: int. Length of the node (in terms of nucleotides, real or simulated).
    `uniqueNodePathsIDs`: numpy.array. A 1D array containing all path IDs (0 based, non-repeated) 
                          that contains the node.
    `pathNodeCount`: numpy.array.: A 1D array (the same length as `uniqueNodePathsIDs`) which 
                     contains number of times the node appear in each path from `uniqueNodePathsIDs`,
                     e.g. if `pathNodeCount[i]` is `k` then node with ID nodeIdx appears in path
                     `uniqueNodePathsIds[i]` `k` times.
    
    '''
    
    # Find positions of the node in each path
    nodePathsIdx, nodeSeqInPath = np.where(pathNodeArray == nodeIdx)
    # Find node length
    nodeLen = nodeLengths[nodeIdx-1]

    # Find unique path IDs (numerical) and cound of given node in each path.
    uniqueNodePathsIDs,pathNodeCount = np.unique(nodePathsIdx,return_counts=True)
    
    return nodePathsIdx,nodeSeqInPath,nodeLen,uniqueNodePathsIDs,pathNodeCount

In [ ]:
#| export
def finaliseComponentBase(component,
                      components,componentNucleotides,
                      matrix,occupants,nBins,componentLengths,nucleotides,zoomLevel,accessions,inversionThreshold=0.5):
        
    componentLengths.append(nBins)
    component["matrix"].extend([[pathID,
                                 int(matrixPathArray[1][0][1]>inversionThreshold),
                                 matrixPathArray] \
                                for pathID,matrixPathArray in matrix.items()])
    component['binsToCols'] = [1]*nBins
    
    component["occupants"] = sorted(list(occupants))
    component['last_bin'] = component['first_bin'] + nBins - 1
    component['lastCol'] = component['firstCol'] + nBins - 1
    component['binColStarts'] = list(range(component['firstCol'],component['lastCol']+1))
    component['binColEnds'] = list(range(component['firstCol'],component['lastCol']+1))
    if nucleotides!='':
        componentNucleotides.append(nucleotides)
    
    firstBin = component['last_bin'] + 1
    firstCol = component['lastCol'] + 1
    components.append(component)
    component = deepcopy(componentTemplate)
    component['first_bin'] = firstBin
    component['firstCol'] = firstCol
    return component,components,componentNucleotides,{},set(),0,''

In [ ]:
#| export
def processAnnotationInterval(posStart,posEnd,annotation,res):
    for pos in range(posStart,posEnd):
        r = []
        for name,intervals in annotation.items():
            for interval in intervals:
                if pos<=interval[1] and pos>=interval[0]:
                    r.append(name)
        res[pos] = r

In [ ]:
#| export
def combineAnnotation(combAnnotation):
    for accAnn in combAnnotation.values():
        for annString,intList in accAnn.items():
            accAnn[annString] = combineIntervals(intList)

In [ ]:
#| export
def updateEdges(accEdge,edgeAccessions,compNum):
    '''
    Function fills up either accStarts or accEnds (on which component each accession starts and 
    on which ends). `compNum` is assumed to be 1-based.
    '''
    
    for accID in edgeAccessions:
        accEdge[accID] = compNum
        
    return accEdge

### Wrapper

Now 'positions' key in metadata contains either one position (chr:posStart..posEnd) or two comma separated positions where one is genomic position, and another one is pangenomic position.

In [ ]:
#| export
def baseLayerZoom(graph,
                  outputPath,outputName,
                  pathNodeArray,pathDirArray,
                  pathLengths,nodeLengths,
                  pathNodeLengthsCum,
                  maxLengthComponent,
                  blockEdges,
                  CPUS = cpu_count(),
                  inversionThreshold=0.5,
                  isSeq=True,
                  debug=False,
                  debugTime=False):

    zoomLevel = 1
    
    print('\n===========================')
    print(f'Zoom level {zoomLevel}')
    print('===========================')
    zoom_level_struct = {}
    zoom_level_struct["files"] = []
    
    accStarts = {}
    accEnds = {}
    
    invertedStarts = set()
    invertedEnds = set()
    
    nodeToComponent = []
    componentToNode = []
    componentLengths = []
    components = []
    componentNucleotides = []

    component = deepcopy(componentTemplate)
    
    numNodes = len(graph.nodes)
    numNodesDigits = int(np.ceil(np.log10(numNodes)))
    
    fromLinks = {}
    toLinks = {}
    
    fromComponentLinks = {}
    toComponentLinks = {}
    
    occupants = set()
    nucleotides = ''
    matrix = {}

    nodeLinks = []

    nBins = 0
    binLength = 0
    occupancy = {}
    inversion = {}
    pos = {} #???
    nodesInComp = set()
    annotationNames = {}
    
    combAnnotation = {}
    combGenPos = {}
    combGenPosSearch = {}
    combPangenPosSearch = {}
    combAltChrGenPos = {}
    annFirstBin = 0
    
    for nodeIdx in range(1,numNodes+1):
        if debug or debugTime:
            print(f'Processing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}')
        else:
            print(f'\rProcessing node {nodeIdx:0{numNodesDigits}}/{numNodes:0{numNodesDigits}}',end='')
        
        if debugTime:
            startNodeTime = time.time()
        ######################
        # Node preprocessing #
        ######################
        # Get some onfo about the node
        nodePathsIdx,nodeSeqInPath,nodeLen,uniqueNodePathsIDs,pathNodeCount = \
        nodeStat(nodeIdx,pathNodeArray,nodeLengths)
        
        # Combining annotations
        for accID,accNodeAnn in graph.nodesMetadata[nodeIdx-1].items():    
            for annString,intervals in accNodeAnn['annotation'].items():
                for interval in intervals:
                    combAnnotation.setdefault(accID,{}).\
                    setdefault(annString,[]).\
                    append((interval[0]+annFirstBin,interval[1]+annFirstBin))
            for genPos in accNodeAnn.get('positions',[]): #switch to 'positions' key and include pangenome coordinates
                genposstring = ''
                if 'gen' in genPos:
                    genPosString += f'{genPos["gen"]["chr"]}:{"..".join(map(str,genPos["gen"]["genomePosition"]))}'
                    combGenPosSearch.setdefault(accID,{}).setdefault(f'{annFirstBin}_{genPos["gen"]["chr"]}',[]).append(genPos["gen"]["genomePosition"])
                if 'pangen' in genPos:
                    genPosString += f'|{genPos["pangen"]["chr"]}:{"..".join(map(str,genPos["pangen"]["genomePosition"]))}'
                    combPangenPosSearch.setdefault(accID,{}).setdefault(f'{annFirstBin}_{genPos["pangen"]["chr"]}',[]).append(genPos["pangen"]["genomePosition"])
                combGenPos.setdefault(accID,{})[genPosString]=[(annFirstBin,annFirstBin+nodeLengths[nodeIdx-1]-1)]
            for altGenPos in accNodeAnn.get('altChrGenPos',[]):
                combAltChrGenPos.setdefault(accID,{})[f'{altGenPos["chr"]}:{"..".join(map(str,altGenPos["genomePosition"]))}']=[(annFirstBin,annFirstBin+nodeLengths[nodeIdx-1]-1)]
        annFirstBin +=nodeLengths[nodeIdx-1]
        
        # Check whether the component should be broken before and/or after current node
        (leftFarLink,rightFarLink,
        leftPathStartEnd,
        rightPathStartEnd,
        isChangedStructure,
        isNotFitIntoComponent,
        pathStarts,pathEnds,
        fromLinks,toLinks,
        occupancy,inversion,
        invertedCompStarts,invertedCompEnds) = checkForBreak(nodeIdx,nodeLen,
                                             nodePathsIdx,nodeSeqInPath,
                                             uniqueNodePathsIDs,pathNodeCount,
                                             pathLengths,
                                             pathNodeArray,pathDirArray,
                                             occupancy,inversion,
                                             fromLinks,toLinks,
                                             nBins,maxLengthComponent,
                                             blockEdges,
                                             inversionThreshold=inversionThreshold,
                                             debug=debug)
        
        invertedStarts.update(invertedCompStarts)
        invertedEnds.update(invertedCompEnds)
        
        if debugTime:
            print(f'Checking for breaks node {nodeIdx}. Took {time.time() - startNodeTime}')
        ###################################
        # Breaking component before node. #
        ###################################
        ## Here component should be broken before node (if determined necessary)
        if nodeIdx>1 and \
           (leftPathStartEnd or leftFarLink or isChangedStructure or isNotFitIntoComponent) and \
           nBins>0:
            # It is not the first component, and there is at least one flag to break before node and 
            # there is something to break (the component was not broken after previous node)
            if debug:
                if leftFarLink:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to left far link.')
                elif isChangeOccupancy or isChangeInversion:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to change in inversion or occupancy in a path.')
                elif isNotFitIntoComponent:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} because it does not fit into current component.')
                else:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to start or end of a path.')

            [component,components,
            componentNucleotides,
            matrix,occupants,
            nBins,
            nucleotides] = finaliseComponentBase(component,components,
                                            componentNucleotides,
                                            matrix,occupants,
                                            nBins,
                                            componentLengths,
                                            nucleotides,
                                            zoomLevel=zoomLevel,
                                            accessions=graph.accessions)
            if len(components) not in nodeToComponent[-1]:
                nodeToComponent[-1].append(len(components))
            componentToNode.append(list(nodesInComp))
            nodesInComp = set()
        
        occupants |= set(uniqueNodePathsIDs)
        nodesInComp.add(nodeIdx)
        nodeToComponent.append([])
        
        
        #######################################################
        # Adding bins and forming components for zoom level 1 #
        #######################################################
        # TODO: This process for each bin should be parallelised
        
        # Building nucleotide (real or dummy) for the component.
        if isSeq:
            nucleotides += graph.nodesData[nodeIdx-1]

        # binLength is removed as this function process only single nucleotide level.
        
        # Preparing annotation for each node and each accession
        curNodeAnnotation = {}
        
        forwardPaths = []
        invertedPaths = []
        
        for j,pathID in enumerate(uniqueNodePathsIDs):
            if debugTime:
                startTime = time.time()
            
            pathName = graph.accessions[pathID]
            
            # get occupancy and inversion for current node in current path.
            occ = occupancy[pathID]
            inv = inversion[pathID]
            
            if inv>inversionThreshold:
                invertedPaths.append(pathID)
            else:
                forwardPaths.append(pathID)
            
            # Creating addition to matrix from this node for this path.
            matrixPath = matrix.get(pathID,[[],[]])
            
            
            # Absolute positinal genomic coordinate for given path for each pass of the node.
            # NodeStarts are 1-based (position of the first node in the path will be 1).
            nodeStarts = np.array([pathNodeLengthsCum[pathID,nodeNumInPath]-nodeLen+1 \
                          for nodeNumInPath in nodeSeqInPath[np.where(nodePathsIdx==pathID)[0]]])
            
            
            # Generating positional pairs for each nucleotide of the node.
            posPath = [list(zip(nodeStarts+pos,nodeStarts+pos)) for pos in range(nodeLen)]
            
            if debugTime:
                print(f'Preparation for node {nodeIdx} path {pathName} took {time.time() - startTime}')
                annotationTime = time.time()
            
            if debugTime:
                print(f'Annotations for node {nodeIdx} path {pathName} finished in {time.time()-annotationTime}, overall time {time.time()-startTime}')
                matrixTime = time.time()
            
            matrixPath[0].extend(range(nBins,nBins+nodeLen))
            matrixPath[1].extend(zip([occ]*nodeLen,[inv]*nodeLen,posPath))
            
            matrix[pathID] = matrixPath
            
            if debugTime:
                print(f'MatrixPath for node {nodeIdx} path {pathName} finished in {time.time()-matrixTime}, overall time {time.time()-startTime}')
        
        forwardPaths.extend(set(range(len(graph.accessions))).difference(forwardPaths+invertedPaths))
        
        # After all bins are formed, they should be split among components.
        # If there are more than one component from the node, then the last component 
        # should remain open and the rest of checks should go as normal.
        # This will allow to attach a small fully followed node after 
        # (which should be the single node but due to graph construction error was separated).
        # Otherwise, if only one component is formed, then the normal checks should follow.
        nBinsReduction = 0
        if nodeLen>maxLengthComponent:
            if debugTime:
                postBinTime = time.time()
            for blockStart in range(0,nodeLen+1-maxLengthComponent,maxLengthComponent):
                blockEnd = blockStart + maxLengthComponent
                if len(forwardPaths)>0:
                    addLink(len(components)+1,'+',len(components)+2,'+',forwardPaths,fromComponentLinks,toComponentLinks)

                if len(invertedPaths)>0:
                    addLink(len(components)+2,'-',len(components)+1,'-',invertedPaths,fromComponentLinks,toComponentLinks)

                if debug:
                    print(f'Node {nodeIdx}: Component broken inside node {nodeIdx} due to max component length.')
                matrixSlice = {}
                for pathID,matrixPath in matrix.items():
                    matrix[pathID][0] = list(range(nodeLen-blockEnd))
                    if inversion[pathID]>inversionThreshold:
                        matrixDataSlice = matrixPath[1][-maxLengthComponent:]
                        matrix[pathID][1] = matrixPath[1][:-maxLengthComponent]
                    else:
                        matrixDataSlice = matrixPath[1][:maxLengthComponent]
                        matrix[pathID][1] = matrixPath[1][maxLengthComponent:]
                    matrixSlice[pathID] = [list(range(len(matrixDataSlice))),matrixDataSlice]
                
                if inversion[pathID]>inversionThreshold:
                    nucleotideSlice = nucleotides[-maxLengthComponent:]
                    nucleotides = nucleotides[:-maxLengthComponent]
                else:
                    nucleotideSlice = nucleotides[:maxLengthComponent]
                    nucleotides = nucleotides[maxLengthComponent:]
                
                component,components,componentNucleotides,_,_,_,_ = \
                    finaliseComponentBase(component,components,componentNucleotides,
                                      matrixSlice,occupants,
                                      maxLengthComponent,componentLengths,nucleotideSlice,
                                      zoomLevel=zoomLevel,accessions=graph.accessions)

                nodeToComponent[-1].append(len(components))
                componentToNode.append(list(nodesInComp))
                nodesInComp = set([nodeIdx])
                
                nBinsReduction += maxLengthComponent
            if debugTime:
                print(f'Node splitting for node {nodeIdx} took {time.time()-postBinTime}')
        nBins += nodeLen-nBinsReduction
        
        
        if debugTime:
            postBinTime = time.time()
        #########################################
        # Breaking component after current node #
        #########################################
        # If any path ends on current node, this should be recorded in the component.
        if len(pathEnds)>0:
            if nBins>0:
                component.setdefault("ends",set()).update(pathEnds.tolist())
                accEnds = updateEdges(accEnds,pathEnds,len(components)+1)
            else:
                components[-1].setdefault("ends",set()).update(pathEnds.tolist())
                accEnds = updateEdges(accEnds,pathEnds,len(components))
                
        if len(pathStarts)>0:
            if nBins>0:
                component.setdefault("starts",set()).update(pathStarts.tolist())
                accStarts = updateEdges(accStarts,pathStarts,len(components)+1)
            else:
                components[-1].setdefault("starts",set()).update(pathStarts.tolist())
                accStarts = updateEdges(accStarts,pathStarts,len(components))
        
        if nodeIdx==len(graph.nodes) and nBins>0:
            # Last node in graph.
            if debug:
                print(f'Node {nodeIdx}: Last node in the last component.')
            component,components,componentNucleotides,matrix,occupants,nBins,nucleotides = \
                finaliseComponentBase(component,components,componentNucleotides,matrix,occupants,nBins,componentLengths,nucleotides,
                                  zoomLevel=zoomLevel,accessions=graph.accessions)
            nodeToComponent[-1].append(len(components))
            componentToNode.append(list(nodesInComp))
            nodesInComp = set()

        elif (rightFarLink or rightPathStartEnd) and nBins>0:
            if debug:
                if rightFarLink:
                    print(f'Node {nodeIdx}: Component broken after node {nodeIdx} due to right far link.')
                else:
                    print(f'Node {nodeIdx}: Component broken before node {nodeIdx} due to start or end of a path.')
            component,components,componentNucleotides,matrix,occupants,nBins,nucleotides = \
                finaliseComponentBase(component,components,componentNucleotides,matrix,occupants,nBins,componentLengths,nucleotides,
                                  zoomLevel=zoomLevel,accessions=graph.accessions)
            nodeToComponent[-1].append(len(components))
            componentToNode.append(list(nodesInComp))
            nodesInComp = set()
        # Not sure why this is needed. It will add com
        elif nBins>0:
            nodeToComponent[-1].append(len(components)+1)
        
        if debugTime:
            print(f'After node {nodeIdx} component breaking took {time.time() - postBinTime}')
            print(f'Processing of node {nodeIdx} took {time.time() - startNodeTime}')
    
    combineAnnotation(combAnnotation)
    
    return (numNodes, # number of nodes
            numNodesDigits, # 
            nodeToComponent,
            componentToNode,
            componentLengths,
            components,
            componentNucleotides,
            fromLinks, toLinks,
            fromComponentLinks, toComponentLinks,
            accStarts, accEnds,
            invertedStarts, invertedEnds,
            combAnnotation, combGenPos, combAltChrGenPos, combGenPosSearch, combPangenPosSearch)

## Transfer from nodes to components (links and other structures)

This is one of the first processes happening while exporting graph. While graph operates with nodes (which can be linearly connected with each other in all paths),
then exporting works with components. In almost all cases, components have at least some non-linear links with other components on both sides. The only exclusion is when a component is too large and split into several ones. In this case two components will be connected by 100% linear links. Also, graph operates with paths along with nodes, whereas exporting works with components and accession-specific links between them.

These functions (with main orchestrating one is `nodeToComponentLinks`) are converting nodes and paths to components and links.

In [ ]:
#| export
def splitforwardInversedNodeComp(pathList,component,isInverse):
    forward = []
    inversed = []
    
    for pathID in pathList:
            try:
                if component["matrix"][component["occupants"].index(pathID)][1]>0:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
            except (IndexError,ValueError):
                # If it is artificial pass link.
                if isInverse:
                    inversed.append(pathID)
                else:
                    forward.append(pathID)
                
    return forward,inversed

In [ ]:
#| export
def fillLinksBase(nodeInComp,nodeToComponent,fromLinks,toLinks,fromComponentLinks,toComponentLinks,compNum,components,doLeft=True,doRight=True):
    component = components[compNum]
    for node,isFirstNode in zip([nodeInComp[0],nodeInComp[-1]],[True,False]):
        # Processing all external arrival links
        nodeToLink = toLinks.get(node,{})
        for fromNode in nodeToLink.keys():
            intermediateCondition = (node<fromNode)
            
            la,ra = splitforwardInversedNodeComp(nodeToLink[fromNode],component,intermediateCondition)
            
            fromFirstCompNum = nodeToComponent[fromNode-1][0]
            fromFirstComp = components[fromFirstCompNum-1]
            fromLastCompNum = nodeToComponent[fromNode-1][-1]
            fromLastComp = components[fromLastCompNum-1]
            
            if isFirstNode:
                #left arrivals
                if len(la)>0 and doLeft:
                    frd,fld = splitforwardInversedNodeComp(la,fromFirstComp,intermediateCondition)

                    # from right departure
                    if len(frd)>0 and (fromNode not in nodeInComp) and (fromLastComp['last_bin']+1!=component['first_bin']):
                        addLink(fromLastCompNum,'+',compNum+1,'+',frd,fromComponentLinks,toComponentLinks)
                    #from left departure
                    if len(fld)>0 and (fromNode not in nodeInComp):
                        addLink(fromFirstCompNum,'-',compNum+1,'+',fld,fromComponentLinks,toComponentLinks)
            
            if not isFirstNode:
                #right arrivals
                if len(ra)>0 and doRight:
                    frd,fld = splitforwardInversedNodeComp(ra,fromFirstComp,intermediateCondition)

                    #from right departures
                    if len(frd)>0 and (fromNode not in nodeInComp):
                        addLink(fromLastCompNum,'+',compNum+1,'-',frd,fromComponentLinks,toComponentLinks)
                    #from left departures
                    if len(fld)>0 and (fromNode not in nodeInComp):
                        addLink(fromFirstCompNum,'-',compNum+1,'-',fld,fromComponentLinks,toComponentLinks)
            
        # Processing all external departure links
        nodeFromLink = fromLinks.get(node,{})
        for toNode in nodeFromLink.keys():
            intermediateCondition = (node>toNode)
            
            rd,ld = splitforwardInversedNodeComp(nodeFromLink[toNode],component,intermediateCondition)
            
            toFirstCompNum = nodeToComponent[toNode-1][0]
            toFirstComp = components[toFirstCompNum-1]
            toLastCompNum = nodeToComponent[toNode-1][-1]
            toLastComp = components[toLastCompNum-1]
            
            if not isFirstNode:
                #right departures
                if len(rd)>0 and doRight: # Check if doRight is set incorrectly for our case (121->122 at level 4)
                    tla,tra = splitforwardInversedNodeComp(rd,toFirstComp,intermediateCondition)

                    #to left arrivals
                    if len(tla)>0:
                        addLink(compNum+1,'+',toFirstCompNum,'+',tla,fromComponentLinks,toComponentLinks)

                    #to right arrivals
                    if len(tra)>0: # Most probably the problem is here! Check it!
                        addLink(compNum+1,'+',toLastCompNum,'-',tra,fromComponentLinks,toComponentLinks)
            
            if isFirstNode:
                #left departures
                if len(ld)>0 and doLeft:
                    tla,tra = splitforwardInversedNodeComp(ld,toFirstComp,intermediateCondition)

                    #to left arrivals
                    if len(tla)>0:
                        addLink(compNum+1,'-',toFirstCompNum,'+',tla,fromComponentLinks,toComponentLinks)

                    if len(tra)>0:
                        addLink(compNum+1,'-',toLastCompNum,'-',tra,fromComponentLinks,toComponentLinks)
                        
    return fromComponentLinks,toComponentLinks

In [ ]:
#| export
def convertLink(linkFrom,linkTo,translateDict,forwardLinks,isZoom):
    
    linkSetConv = set()
    if isZoom:
        linkFound = False
        for fromStrand,fromStrandDict in forwardLinks[linkFrom].items():
            for toStrand,accessionList in fromStrandDict.get(linkTo,{}).items():
                linkFromConv = translateDict[linkFrom-1][0 if fromStrand=='-' else -1]+1
                linkToConv = translateDict[linkTo-1][0 if toStrand=='+' else -1]+1
                linkSetConv.add((linkFromConv,linkToConv))
                linkFound = True
        if not linkFound:
            warnings.warn(f'Link ({linkFrom},{linkTo}) is requested to convert, but was not found in forwardLinks!',
                          category=RuntimeWarning)
    else:
        for fromStrand,fromStrandPairs in forwardLinks[linkFrom].items():
            for toNode,toStrand in fromStrandPairs:
                if toNode==linkTo:
                    linkFromConv = translateDict[linkFrom-1][0 if fromStrand=='-' else -1]
                    linkToConv = translateDict[linkTo-1][0 if toStrand=='+' else -1]
                    linkSetConv.add((linkFromConv,linkToConv))
                    break
        
    return linkSetConv

In [ ]:
#| export
def recordUpdatedPairedLink(firstLinkSet,secondLinkSet,firstLink,secondLink,substituteLink,pairedLinksConv):
    if substituteLink==(firstLink[0],secondLink[1]):
        # Straight paired link of inverted double paired link
        for fLink in firstLinkSet:
            for sLink in secondLinkSet:
                pairedLinksConv.setdefault(fLink,{})[sLink] = (fLink[0],sLink[1])
        
    elif substituteLink==(secondLink[0],firstLink[1]):
        # inverted paired link or straight doublePaired link
        
        for fLink in firstLinkSet:
            for sLink in secondLinkSet:
                pairedLinksConv.setdefault(fLink,{})[sLink] = (sLink[0],fLink[1])
    else:
        raise RuntimeError(f'Unusual paired link encountered {firstLink}+{secondLink} -> {substituteLink}')
    

In [ ]:
#| export
def convertRemovableComponents(translateDict,linkLengths,pairedLinks,interconnectedLinks,blockEdges,forwardLinks,isZoom=True):
    '''
    translateDict should be a dict in format {<old node/component id 0-based>:<new component id 1-based>}
    pathNodeInv should be a dict of dicts of the following structure:
    {<pathID>:{<nodeId 1-based>:<Bool inv or not>}}
    
    This is done through fromLinks and toLinks and throught associated directions of available accessions.
    For this we need to loop through strands and do it separately for each strand.
    
    For paired links there is a possibility that a single node link will give several component links. 
    In this case, the cross product of all first and second links will be added to converted paired links.
       
    ❗The substitute links should be added only to the paths that contained both first and second links in the first place.
        This should be controlled in link removal routine.
    
    '''
    if linkLengths is None or pairedLinks is None or interconnectedLinks is None or blockEdges is None:
        return None,None,None,None
    processedLinks = {}
    print('Converting link to block lengths associations')
    linkLengthsConv = {}
    for blockLength,linkList in linkLengths.items():
        for linkFrom,linkTo in linkList:
            linksConv = convertLink(linkFrom,linkTo,translateDict,forwardLinks,isZoom)
            linkLengthsConv.setdefault(blockLength,set()).update(linksConv)
            for link in linksConv:
                processedLinks.setdefault(link,set()).add(blockLength)

    print('Converting paired links')
    pairedLinksConv = {}
    for firstLink,flDict in pairedLinks.items():
        firstLinkSet = convertLink(*firstLink,translateDict,forwardLinks,isZoom)
        for secondLink,substituteLink in flDict.items():
            secondLinkSet = convertLink(*secondLink,translateDict,forwardLinks,isZoom)
            recordUpdatedPairedLink(firstLinkSet,secondLinkSet,firstLink,secondLink,substituteLink,pairedLinksConv)
    
    print('Converting interconnected links')
    interconnectedLinksConv = {}
    for link,linkList in interconnectedLinks.items():
        convertLinks = convertLink(*link,translateDict,forwardLinks,isZoom)
        for linkC in convertLinks:
            
            linkListConv = interconnectedLinksConv.get(linkC,set())
            originalLinkListLen = len(linkListConv)
            
            for linkI in linkList:
                linkListConv.update(convertLink(*linkI,translateDict,forwardLinks,isZoom))
            
            if originalLinkListLen>0:
                # join two blocks
                processedLinks.pop(linkC,None)
                linkListConv.add(linkC)
                blockLen = []
                for blLen,linkList in list(linkLengthsConv.items()):
                    if linkC in linkList:
                        linkLengthsConv[blLen] = linkList - linkListConv
                        blockLen.append(blLen)
                        
                maxBlockLen = max(blockLen)
                linkLengthsConv[maxBlockLen].update(linkListConv)
                
                for link in linkListConv:
                    interconnectedLinksConv[link] = linkListConv
            else:
                interconnectedLinksConv[linkC] = linkListConv
    
    for link,blList in processedLinks.items():
        if len(blList)>1:
            for blockLength in blList:
                linkLengthsConv[blockLength].remove(link)
            linkLengthsConv[max(blList)].add(link)
    
    print('Converting rearrangement blocks')
    blockEdgesConv = {translateDict[node-1][-1]:blockLength for node, blockLength in blockEdges.items()}
    
    return linkLengthsConv,pairedLinksConv,interconnectedLinksConv,blockEdgesConv

In [ ]:
#| export
def nodeToComponentLinks(components,componentToNode,nodeToComponent,
                         fromLinks,toLinks,graph,
                         fromComponentLinks,toComponentLinks,
                         linkLengths=None,pairedLinks=None,interconnectedLinks=None,blockEdges=None,
                         debug=False):
    numComps = len(components)
    numCompsDigits = int(np.ceil(np.log10(numComps)))
    
    for compNum in range(numComps):
        if debug:    
            print(f'Processing component links {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}')
        else:
            print(f'\rProcessing component links {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}',end='')

        component = components[compNum]
        
        nodeInComp = componentToNode[compNum]
        nodeInComp.sort()

        if len(nodeInComp)>1:
            fromComponentLinks,toComponentLinks = fillLinksBase(nodeInComp,nodeToComponent,fromLinks,toLinks,fromComponentLinks,toComponentLinks,compNum,components)
        elif len(nodeInComp)==1:

            mainNode = nodeInComp[0]
            doLeft = False
            if compNum==0:
                doLeft = True
            elif mainNode not in componentToNode[compNum-1]:
                doLeft = True

            doRight = False # Check this conditions as they may be causing issue!
            if compNum==(len(components)-1):
                doRight = True
            elif mainNode not in componentToNode[compNum+1]:
                doRight = True

            fromComponentLinks,toComponentLinks = fillLinksBase(nodeInComp,nodeToComponent,fromLinks,toLinks,fromComponentLinks,toComponentLinks,compNum,components,doLeft,doRight)

        else:
            raise ValueError(f"Component {compNum} does not have any associated nodes!")
    
    print('')
    
    linkLengthsConv,pairedLinksConv,interconnectedLinksConv,blockEdgesConv = \
    convertRemovableComponents(nodeToComponent,linkLengths,pairedLinks,interconnectedLinks,blockEdges,graph.forwardLinks,isZoom=False)
    
    return fromComponentLinks,toComponentLinks,linkLengthsConv,pairedLinksConv,interconnectedLinksConv,blockEdgesConv

## Identifying collapsible links and rearrangement blocks (works incorrectly, left now for compatibility).

In order to be able to generate multiple zoom levels of the graph view, non-linear links describing small (too small to show at the given 
zoom level) rearrangements should disappear whereas links describing larger blocks should persist. This will allow to see larger rearrangements clearly
on higher zoom levels.

In order to do it, each link should be associated with some size (or rearrangement), so, that when each zoom level is generated, they can be removed when the rearrangement cannot be shown at the given zoom level.

Some links are also associated with each other, and when they are removed new links (usually linear ones) should be reinstated to make larger rearrangements clearer.

At the moment, the process of identifying these sizes is not working great as it leaves too much non-linear links to the very top level where suddenly all non-linear links disappear and the whole graph from over-complicated jumps to pretty much trivial without any rearrangements. If to use digital map analogy, most of country roads persist while you zoom out on the map until almost the whole Earth is in view and then at some point the view becomes just a blue/green ball with very rough boundary of continents and oceans.

At the moment, all associated links get into a pool of so called interconnected links and if one link gets associated with specific size, then all links get the same association, and then maximum size is selected. But that means that if one link describes one small and also on the edge of large rearrangement, and another link is only associated with large rearrangement, then the latter link will also be associated with the size of large rearrangement and will stay until the zoom level where the large rearrangement is too small to show. That is incorrect. 

I think, each link should get its own associations with sizes (and maximum should be taken) and clearing of the link should happen individually. Yet, if one link with smaller size and one with larger size are paired, the reinstated link should appear after smaller link removed.

Another alternative is just to get contiguous blocks in each path and associate each link pair (describing start and end of each block) as a pair of links that needs to be cleared in association with the size of this block. Need control of repeats in these blocks. If it happens, then a single link can describe a whole rearrangement. In addition, an extra control for inversion is also needed. In particular, if outside the block the numbers do not create a range to fin the inverted node (e.g. 1+,4-,3+, or 3+,2-,5+), then it should be ignorred for this step. It means there is a smaller rearrangement within larger one.

Another alternative (described in TODO) is to convert paths of nodes to paths of edges and operate with them. I guess, it is not far away from the previous paragraph.

### Identifying path breaks

In [ ]:
#| export
def findBreaksInPath(combinedArray,nextNodeDict):
    return [pos for pos,node in enumerate(combinedArray[:-1]) \
                      if (node<0 and combinedArray[pos+1]!=-1*nextNodeDict.inverse.get(-1*node,[-1*node-1])[0]) or\
                         (node>0 and combinedArray[pos+1]!=nextNodeDict.get(node,node+1))]

In [ ]:
#| export
def identifyPathBreaks(combinedNodeDirArray,pathLengths,pathNextNode):
    pathBreakCoordPairs = []
    for pathID in range(combinedNodeDirArray.shape[0]):
        # separate this block into separate function as it will be used in link processing!
        tempBreaks = [(pathID,pos) for pos in findBreaksInPath(combinedNodeDirArray[pathID,:pathLengths[pathID]], pathNextNode[pathID])]
                      
        pathBreakCoordPairs.extend(tempBreaks)
    
    return pathBreakCoordPairs

### Block processing

In [ ]:
#| export
def interweaveArrays(a,b):
    if len(a.shape)>1 or len(b.shape)>1:
        raise ValueError('Both input arrays should be 1D arrays')
        
    if a.shape[0]!=b.shape[0]:
        raise ValueError('Both input arrays should have the same lengths')
        
    c = np.empty((a.size + b.size,), dtype=a.dtype)
    c[0::2] = a
    c[1::2] = b
    
    return c

In [ ]:
#| export
def extractGapsBlocks(block,path,nodeLengths,getComplex=False):
    '''
    This function either split block by gaps (e.g. block [1,2,4,5,6,8] will yield [1,2],[4,5,6],[8])
    
    If `getComplex` is set to True, then first gaps are filtered for nodes that are not passed by the path. 
    After that, edges are identified and then for them nodes not passed by the path are filtered out.
    Then we find the longest block out of edges, and then the longest edge combine with all gaps and find 
    the shortest one. That shortest one is going to be the one returned.
    
    E.g. block [1,2,4,5,8] will give edges [1,2],[4,5],[8] and gaps [3],[6,7].
    
    If path does not contain 6, then edges will be the same, but gaps will be [3],[6]
    
    If path does not contain 3, then edges will be [1,2,4,5],[8] and gaps [6,7]
    
    The exact block which will be returned depends on sizes of each node.
        
    Return
    ======
    
    List of blocks and whether any gaps are identified or not
    '''
    
    gapList = np.where(np.diff(np.sort(block))!=1)[0]
    
    gapPosInterweaved = interweaveArrays(gapList,gapList+1)

    gapEdgeNodes = np.array(block)[gapPosInterweaved]

    gapEdgeNodes[1::2] = gapEdgeNodes[1::2]-1
    
    gaps = np.split(np.arange(1,max(path)+1),gapEdgeNodes)[1::2]
    
    if len(gaps)>0:
        if getComplex:
            # It looks like it is possible to identify edges first and then filter gaps for nodes not used by the path
            # But that will create edges that contain nodes that are not used by the path, which still bring us to two filtering.
            
            # Removing parts (or whole) of gaps, which are not actually gaps, but nodes never used by the path.
            gaps = list(filter(lambda bl: len(bl)>0,[tuple((gap[np.isin(gap,path)]).tolist()) for gap in gaps]))
            
            # Removing parts of the edges that turned out to contain nodes not passed by the path
            edges = list(filter(lambda bl: len(bl)>0,[tuple((bl[np.isin(bl,path)]).tolist()) for bl in np.split(block,gapList+1)]))

            edgesLengths = [sum([nodeLengths[node-1] for node in bl]) for bl in edges]
            maxEdgeInd = np.argmax(edgesLengths)
            maxEdge = edges[maxEdgeInd]
            maxEdgeLength = edgesLengths[maxEdgeInd]
            
            gapLengths = [sum([nodeLengths[node-1] for node in bl]) for bl in gaps]
            gaps += [maxEdge]
            gapLengths += [maxEdgeLength]
            
            minBlockInd = np.argmin(gapLengths)
            minBlock = gaps[minBlockInd]
            return [tuple(minBlock)],True
        else:
            return [tuple(bl) for bl in np.split(block,gapList+1)],True
    else:
        return [tuple(block)],False#checkSplitBlock(block)

In [ ]:
#| export
def checkSplitBlock(block,gapList=None):
    '''
    Not used at the moment
    
    Function checks if the block has any gaps and split into a list of blocks between gaps 
    (alternatively fill gaps or leave things as they are).
    At the moment the gapped block will be converted to list of blocks between gaps
    
    Parameters
    ==========
    
    `block`: np.array 1D. An array of node ids conprising the block.
    
    Return
    ======
    `blockList`: list[np.array]. A list of numpy arrays of eligible blocks.
    '''
    if gapList is None:
        gapList = np.where(np.diff(np.sort(block))!=1)[0]
    
    if len(gapList>0):
        return [tuple(bl) for bl in np.split(block,gapList+1).tolist()]
    else:
        return [tuple(block)]

In [ ]:
#| export
def blockListToLengths(blockList,nodeLengths):
    return [np.sum([nodeLengths[node-1] for node in block]) for block in blockList]

In [ ]:
#| export
def _blockToLengthLink(link,blockList,linkL,nodeLengths):
    linkL.append((link,max(blockListToLengths(blockList,nodeLengths))))

In [ ]:
#| export
def convertBlocksToLengths(linksBlocks,nodeLengths):
    ''' 
    Converting blocks associated with each link to lengths and then selecting the longest one (?)
    '''
    
    p = Pool(cpu_count()-2)
    m = Manager()
    linkL = m.list()
    numLinks = len(linksBlocks)
    for i,_ in enumerate(p.starmap(partial(_blockToLengthLink,linkL=linkL,nodeLengths=nodeLengths),linksBlocks.items()),start=1):
        print(f'\rConverting blocks to block lengths {i}/{numLinks}',end='')
    
    print('\nConversion finished.')
    
    linkLengths = {}
    
    for linkNum,[link,blockSize] in enumerate(linkL,start=1):
        # This option creates a dict <link>:<block length>    
        # linkLengths[link] = max(blockListToLengths(blockList,nodeLengths))
        print(f'\rReformating links to block lengths associations {linkNum}/{numLinks}', end = '')
        # This option creates a dict <block length>:[<link>,<link>,...]
        linkLengths.setdefault(blockSize,set()).add(link)
    
        # The second option will be easier to work with during layers generation.
    print('\nReformating finished.')
    return linkLengths

### Link processing

In [ ]:
#| export
def addToLinkPool(link1,link2,interconnectedLinks):
    # Change signature of the function in the downstream function
    
    interconnectedLinks.setdefault(link1,set()).add(link2)
    interconnectedLinks.setdefault(link2,set()).add(link1)

In [ ]:
#| export
def blockFromSingleLink(pathID,link,pathNodeInversionRate,pathNextNode):
    '''
    Identify block from a single link
    It is the block that the link bounds, i.e.:
    If link if forward then it is inside the link + any side that is inverted
    If link is backward, then it is inside + any side that is normal direction.
    '''
    nodeDir = pathNodeInversionRate[pathID]
    leftNode,rightNode = link
    leftDir = nodeDir.get(leftNode,False)
    rightDir = nodeDir.get(rightNode,False)
    
    if leftNode<rightNode:
        # Next used node after `leftNode`
        blockStart = pathNextNode[pathID].get(leftNode,leftNode+1)
        # Last used node before `rightNode`
        blockEnd = pathNextNode[pathID].inverse.get(rightNode,[rightNode-1])[0]
        block = set(range(blockStart,blockEnd+1))
        if leftDir:
            block.add(leftNode)
        if rightDir:
            block.add(rightNode)
    else:
        # Next used node after `leftNode`
        blockStart = pathNextNode[pathID].get(rightNode,rightNode+1)
        # Last used node before `rightNode`
        blockEnd = pathNextNode[pathID].inverse.get(leftNode,[leftNode-1])[0]
        
        block = set(range(blockStart,blockEnd+1))
        if not leftDir:
            block.add(leftNode)
        if not rightDir:
            block.add(rightNode)
    
    return tuple(block)

In [ ]:
#| export
def checkIndividualLink(link,pathID,usedSecondInPairLink):
    '''
    Function checks if this link is already second in pair. If it is, then it is not considered separately (return True?). 
    Otherwise, it should be considered and block generated (using `blockFromSingleLink`) and associated with this link.
    '''
    return link in usedSecondInPairLink.get(pathID,set())

In [ ]:
#| export
def processDoublePairedLinks(leftLink,rightLink,pathID,doublePairedLinks,pairedLinks,interconnectedLinks,linksBlocks,pathNextNode):
    # Check if it is second link already.
    # If it is, pair second link of the current pair with first link of the previous pair
    # If not, add second link of the current pair with pathID as a key and first link as a value (or list).
    # TBD: Should used links be removed from doublePairedLinks?
    
    pathDouble = doublePairedLinks.get(pathID,{})
    
    if leftLink in pathDouble:
        firstLeftLink = pathDouble[leftLink]
        commonLink = leftLink
        secondRightLink = rightLink
        if pathNextNode[pathID].get(secondRightLink[0],secondRightLink[0]+1)==firstLeftLink[1]:
            pairedLinks.setdefault(secondRightLink,{})[firstLeftLink] = (secondRightLink[0],firstLeftLink[1])
            pairedLinks.setdefault(firstLeftLink,{})[secondRightLink] = (secondRightLink[0],firstLeftLink[1])
        
            addToLinkPool(secondRightLink,firstLeftLink,interconnectedLinks)
    
    
    doublePairedLinks.setdefault(pathID,{})[rightLink] = leftLink
        

In [ ]:
#| export
def processIndividualLink(link,pathID,pathNodeInversionRate,pathNextNode,usedSecondInPairLink):
    if usedSecondInPairLink is not None and checkIndividualLink(link,pathID,usedSecondInPairLink):
        # If usedSecondInPairPath is provided, check if the link was already second in pair. In that case do not do anything.
        return []
    
    block = blockFromSingleLink(pathID,link,pathNodeInversionRate,pathNextNode)
    
    return [block]

In [ ]:
#| export
def recordLinkBlockAssociation(link,blockList,linksBlocks):
    linksBlocks.setdefault(link,set()).update(blockList)

In [ ]:
#| export
def findNextNode(node,combinedArray):
    return np.where(combinedArray==node)[0]

In [ ]:
#| export
def processPseudoPair(breakPos,returnPos,pathID,
                      pathNodeArray,combinedNodeDirArray,pathNextNode,nodeLengths,
                      usedSecondInPairPath,pairedLinks,linksBlocks):
    
    #######Processing pseudo paired links
    mainLink = tuple(pathNodeArray[pathID,breakPos:breakPos+2].tolist())
    dirStart = combinedNodeDirArray[pathID,breakPos]
    # Collect all unique nodes between breakPos+1:returnPos
    # Process block through min(max(edges),gaps) = minBlock
    blockList,isGaps = extractGapsBlocks(np.unique(pathNodeArray[pathID,breakPos+1:returnPos]).tolist(),pathNodeArray[pathID],nodeLengths,getComplex=True)

    # Find the last non-linear link between breakPos+1:returnPos (let's call it (e,f))
    breaksInBlock = findBreaksInPath(combinedNodeDirArray[pathID,breakPos+1:returnPos],pathNextNode[pathID])
    
    ###########
    # When searching for the last non-linear link, do not consider pure change of direction as non-linearity!!!
    # I.e. if the links is (4-,3+) or (2+,3-) they are not non-linear for this purpose
    # Links like (2-,4+) (given that 3 exist in the path). At the same time, I believe (possibly wrongly), 
    # that if the direction for the link start is reversed (e.g. 2-,3+), it should already be considered as non-linear.
    #########################
    
    returnLinkStart = None
    for auxBreak in breaksInBlock[::-1]:
        
        node = pathNodeArray[pathID,auxBreak+breakPos+1]
        nextNode = pathNodeArray[pathID,auxBreak+1+breakPos+1]
        if (dirStart>0 and nextNode!=pathNextNode[pathID].get(node,node+1)) or\
           (dirStart<0 and nextNode!=pathNextNode[pathID].inverse.get(node,node-1)):
            returnLinkStart = auxBreak+breakPos+1
            break
    
    if returnLinkStart is not None:
        lastNonLinearLink = tuple(pathNodeArray[pathID,returnLinkStart:returnLinkStart+2].tolist())

        if not checkIndividualLink(lastNonLinearLink,pathID,usedSecondInPairPath):
            # If (e,f) was not used in pairs before, (?) it should be associated with minBlock
            recordLinkBlockAssociation(lastNonLinearLink,blockList,linksBlocks)
            # (e,f) should be consider as used in pair in the path, but not actually paired!
            usedSecondInPairPath.setdefault(pathID,set()).add(lastNonLinearLink)


    if (isGaps and checkIndividualLink(mainLink,pathID,usedSecondInPairPath)) or mainLink in pairedLinks:
        # If there were gaps and leftLink was second in pair: do not associate leftLink with minBlock
        # Reset blockList, so, nothing will be recorded for the main link
        blockList = []

    ######## End of pseudo pairs processing
    
    return blockList

In [ ]:
#| export
def processStartsEnds(mainLink,linkStarts,linkEnds,interconnectedLinks,forwardLinks):
    '''
    Currently not in use.
    
    TODO!!! Need to add checks for whether one link is intersecting the other or one is fully inside.
    '''
    potentialStartLinks = linkStarts.get(mainLink[0],set())
    potentialEndLinks = linkEnds.get(mainLink[1],set())

    if mainLink not in potentialStartLinks or mainLink not in potentialEndLinks:
        strandSwitch = {'+':'-','-':'+'}
        startStrands = set()
        endStrands = set()
        for startStrand,strandDict in forwardLinks[mainLink[0]].items():
                if (mainLink[1],'+') in strandDict:
                    startStrands.add(startStrand)
                    endStrands.add('+')
                if (mainLink[1],'-') in strandDict:
                    startStrands.add(startStrand)
                    endStrands.add('-')
        
        if mainLink not in potentialStartLinks:
            for link in potentialStartLinks:
                for startStrand in startStrands:
                    toNodeStrandList = forwardLinks[link[0]].get(strandSwitch[startStrand],[])
                    if ((link[1],'+') in toNodeStrandList or (link[1],'-') in toNodeStrandList) and \
                    ((startStrand=='+') and (mainLink[1]>link[1]) or\
                    (startStrand=='-') and (mainLink[1]<link[1])):
                        addToLinkPool(mainLink,link,interconnectedLinks)
            linkStarts.setdefault(mainLink[0],set()).add(mainLink)

        if mainLink not in potentialEndLinks:
            for link in potentialEndLinks:
                for endStrand in endStrands:
                    if ((link[1],strandSwitch[endStrand]) in forwardLinks[link[0]].get('+',[]) or\
                       (link[1],strandSwitch[endStrand]) in forwardLinks[link[0]].get('-',[])) and \
                        ((endStrand=='+') and (mainLink[0]<link[0]) or\
                        (endStrand=='-') and (mainLink[0]>link[0])):
                        addToLinkPool(mainLink,link,interconnectedLinks)
            linkEnds.setdefault(mainLink[1],set()).add(mainLink)

In [ ]:
#| export
def postprocessLinksBlocks(linksBlocks,interconnectedLinks):
    G = nx.Graph(interconnectedLinks)
    
    connLinks = list(nx.connected_components(G))
    numConnLinks = len(connLinks)
    
    for connLinkListNum,connLinkList in enumerate(connLinks):
        print(f'\rPostprocessing interconnected links {connLinkListNum+1}/{numConnLinks}',end='')
        blocks = set().union(*[linksBlocks[link] for link in connLinkList])
        for link in connLinkList:
            interconnectedLinks.setdefault(link,set()).update(connLinkList-set([link]))
            linksBlocks[link] = blocks
    
    print('\nPreprocessing interconnected links finished.')

In [ ]:
#| export
def processPathBreaks(pathBreakCoordPairs,
                       pathNodeArray,pathNextNode,combinedNodeDirArray,pathNodeInversionRate,pathLengths,nodeLengths,forwardLinks):
    linksBlocks = {}
    pairedLinks = {}
    interconnectedLinks = {}
    linkStarts = {}
    linkEnds = {}
    doublePairedLinks = {}
    usedSecondInPairPath = {}
    
    print('Processing path breaks...')
    numPathBreaks = len(pathBreakCoordPairs)
    for breakID,(pathID,breakPos) in enumerate(pathBreakCoordPairs):
        print(f'\rProcessing path break number {breakID+1}/{numPathBreaks}',end='')
        leftLink = (pathNodeArray[pathID,breakPos],pathNodeArray[pathID,breakPos+1])
        
        if combinedNodeDirArray[pathID,breakPos]<0:
            # Inverted node
            nextExpectedNode = -1*pathNextNode[pathID].inverse.get(leftLink[0],[leftLink[0]-1])[0]
        else:
            # normal node
            nextExpectedNode = pathNextNode[pathID].get(leftLink[0],leftLink[0]+1)

        nextPos = findNextNode(nextExpectedNode,combinedNodeDirArray[pathID,breakPos+1:pathLengths[pathID]])+breakPos+1

        blockList = []

        if len(nextPos)>0:
            # There is the node next-in-order after the break
            returnPos = np.sort(nextPos)[0]

            # Is it even possible???
            if returnPos==breakPos+1:
                continue

            rightLink = (pathNodeArray[pathID,returnPos-1],pathNodeArray[pathID,returnPos])
            substituteLink = (pathNodeArray[pathID,breakPos],pathNodeArray[pathID,returnPos])

            # I suspect the first condition cannot be violated by constructtion of the leftLink (always non-linear) 
            # and substitute link (always linear)
            if leftLink!=substituteLink and rightLink!=substituteLink:
                blockList,isGaps = extractGapsBlocks(np.unique(pathNodeArray[pathID,breakPos+1:returnPos]).tolist(),pathNodeArray[pathID],nodeLengths)
                #################################################################################################
                # !!!IMPORTANT!!!: When the link is substituted during generation of next layer, if the new link 
                # is not (k,k+1) (or (k+1,k) for inverted nodes), then for each path it has to be checked whether
                # follow-through links through empty components can be established. Implemented, but need tested.
                #################################################################################################

                # Adding paired links with their substitution
                pairedLinks.setdefault(leftLink,{})[rightLink] = substituteLink
                pairedLinks.setdefault(rightLink,{})[leftLink] = substituteLink
                usedSecondInPairPath.setdefault(pathID,set()).add(rightLink)

                processDoublePairedLinks(leftLink,rightLink,pathID,doublePairedLinks,pairedLinks,interconnectedLinks,linksBlocks,pathNextNode)

                addToLinkPool(leftLink,rightLink,interconnectedLinks)
                recordLinkBlockAssociation(rightLink,blockList,linksBlocks)
            else:
                # Paired link is actually not a paired link. Substituting with one of the pair.
                
                # It is impossible condition. Do I need it here?
                if leftLink==substituteLink:
                    leftLink = rightLink
                
                blockList = processPseudoPair(breakPos,returnPos,pathID,
                                              pathNodeArray,combinedNodeDirArray,pathNextNode,nodeLengths,
                                              usedSecondInPairPath,pairedLinks,linksBlocks)
                    
        else:
            #########################
            # Search for loop position ONLY for main link of the same direction (if both from and to are either forward or inverted)
            # Additional condition is that it should be backward link (constituting possible repeat start)
            # If they are of different direction or not reversed, then treat as single link!
            #########################
            if ((combinedNodeDirArray[pathID,breakPos]>0 and leftLink[0]>leftLink[1]) or \
               (combinedNodeDirArray[pathID,breakPos]<0 and leftLink[0]<leftLink[1])) and \
               np.prod(combinedNodeDirArray[pathID,breakPos:breakPos+2])>0:
                loopPos = findNextNode(combinedNodeDirArray[pathID,breakPos],combinedNodeDirArray[pathID,breakPos+1:pathLengths[pathID]])+breakPos+1
            else:
                loopPos = []

            if len(loopPos)>0:
                # Should happen the same as with pseudo paired links above except all searches should happen in breakPos+1:returnPos+1
                returnPos = loopPos[0]
                blockList = processPseudoPair(breakPos,returnPos+1,pathID,
                                              pathNodeArray,combinedNodeDirArray,pathNextNode,nodeLengths,
                                              usedSecondInPairPath,pairedLinks,linksBlocks)
            else:
            
                # There is no node next-in-order after the break
                # Process a single link as individual
                blockList = processIndividualLink(leftLink,pathID,pathNodeInversionRate,pathNextNode,doublePairedLinks)

                # Finding associated link in path, which are not paired, but interconnected.
                # That is aimed at identifying paired inversed links (start and end of inversed block)
                curLinkFrom = combinedNodeDirArray[pathID,breakPos]
                curLinkFromDir = np.sign(curLinkFrom) # 1 for normal and -1 for inverted
                curLinkFrom *= curLinkFromDir
                curLinkTo = combinedNodeDirArray[pathID,breakPos+1]
                curLinkToDir = np.sign(curLinkTo) # 1 for normal and -1 for inverted
                curLinkTo *= curLinkToDir
                
                if curLinkFromDir!=curLinkToDir:
                    if curLinkFromDir>0:
                        potentialNextFrom = -1*curLinkFromDir*pathNextNode[pathID].get(curLinkFrom,curLinkFrom+1)
                        potentialNextTo = -1*curLinkToDir*pathNextNode[pathID].get(curLinkTo,curLinkTo+1)
                        nextPosFrom = np.where(combinedNodeDirArray[pathID,breakPos:pathLengths[pathID]]==potentialNextFrom)[0]+breakPos
                        nextPosTo = np.where(combinedNodeDirArray[pathID,breakPos+1:pathLengths[pathID]]==potentialNextTo)[0]+breakPos+1
                    else:
                        potentialNextFrom = -1*curLinkFromDir*pathNextNode[pathID].inverse.get(curLinkFrom,[curLinkFrom-1])[0]
                        potentialNextTo = -1*curLinkToDir*pathNextNode[pathID].inverse.get(curLinkTo,[curLinkTo-1])[0]
                        nextPosFrom = np.where(combinedNodeDirArray[pathID,breakPos+1:pathLengths[pathID]]==potentialNextFrom)[0][::-1]+breakPos+1
                        nextPosTo = np.where(combinedNodeDirArray[pathID,breakPos+2:pathLengths[pathID]]==potentialNextTo)[0][::-1]+breakPos+2
                    
                    
                    for posFrom in nextPosFrom:
                        fromLink,toLink = pathNodeArray[pathID,posFrom:posFrom+2]
                        if curLinkFrom+curLinkFromDir==fromLink:
                            addToLinkPool(leftLink,(fromLink,toLink),interconnectedLinks)
                            break
                        
                    for posTo in nextPosTo:
                        fromLink,toLink = pathNodeArray[pathID,posTo-1:posTo+1]
                        if curLinkTo+curLinkFromDir==toLink:
                            addToLinkPool(leftLink,(fromLink,toLink),interconnectedLinks)
                            break
                            
        # processStartsEnds(leftLink,linkStarts,linkEnds,interconnectedLinks,forwardLinks)    
        if len(blockList)>0:
            
            recordLinkBlockAssociation(leftLink,blockList,linksBlocks)
    postprocessLinksBlocks(linksBlocks,interconnectedLinks)
    
    print('\nProcessing path breaks finished.')
    return linksBlocks,pairedLinks,interconnectedLinks

### Rearrangement blocks

In [ ]:
#| export
def addBlockEdge(edge,size,blockEdges):
    curBlockSize = blockEdges.get(edge,0)
    blockEdges[edge] = max(curBlockSize,size)

In [ ]:
#| export
def identifyRearrangementBlocks(nodesStructure,nodeLengths):
    '''
    block Edges is a dict with a structure:
    <edge of block>:<size of block>
    <edge of block> pointing to the node before (!) the break.
    In other words, if it is the start of the block, it will point to the node just before the block,
    and if it is the end of the block, it will point to the last node of the block.
    '''
    
    blockEdges = {}
    
    structQueue = []
    indQueue = []
    nodeNum = len(nodesStructure)
    print('Identifying rearrangement blocks')
    for nodeId,nodeStruct in enumerate(nodesStructure):
        print(f'\rProcessing node {nodeId+1}/{nodeNum}',end='')
        
        try:
            blockStartInd = structQueue.index(nodeStruct)

            blockStart = indQueue[blockStartInd]
            blockEnd = nodeId+1
            if blockStart+1==blockEnd:
                indQueue[-1] = nodeId+1
                continue
            del structQueue[blockStartInd:]
            del indQueue[blockStartInd:]
            blockSize = np.sum([nodeLengths[node-1] for node in range(blockStart+1,blockEnd)])
            addBlockEdge(blockStart,blockSize,blockEdges)
            addBlockEdge(blockEnd-1,blockSize,blockEdges)
        except ValueError:
            pass
        structQueue.append(nodeStruct)
        indQueue.append(nodeId+1)
    
    print('\nIdentifying rearrangement blocks finished.')
    return blockEdges

### Wrapper

In [ ]:
#| export
def getRemovableStructures(graph = None,
                           nodeLengths=None, pathLengths = None, 
                           pathNodeArray = None, pathDirArray = None,
                           pathNextNode = None, forwardLinks = None,
                           inversionThreshold=0.5):
    # Preparing paths for link-block identifications
    
    if nodeLengths is None or pathLengths is None or pathNodeArray is None or pathDirArray is None and pathNextNode is not None or forwardLinks is None:
        if graph is not None:
            forwardLinks = graph.forwardLinks
            
            nodeLengths = calcNodeLengths(graph)

            pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum = initialPathAnalysis(graph,nodeLengths)
        else:
            raise AttributeError(f'Either `graph` or `nodeLengths`, `pathLengths`, `pathNodeArray` and `pathDirArray` should be provided.')

    pathNodeInversionRate,nodesStructure,combinedNodeDirArray = getNodesStructurePathNodeInversionRate(pathNodeArray,pathDirArray,pathLengths,inversionThreshold=inversionThreshold)

    pathNextNode = getNextNodePath(pathNodeArray,pathLengths)
    
    pathBreakCoordPairs = identifyPathBreaks(combinedNodeDirArray,pathLengths,pathNextNode)
    
    linksBlocks,pairedLinks,interconnectedLinks = processPathBreaks(pathBreakCoordPairs,
                      pathNodeArray,pathNextNode,combinedNodeDirArray,pathNodeInversionRate,pathLengths,nodeLengths,forwardLinks)
    
    linkLengths = convertBlocksToLengths(linksBlocks,nodeLengths)
    
    blockEdges = identifyRearrangementBlocks(nodesStructure,nodeLengths)
    
    return linkLengths,pairedLinks,interconnectedLinks,blockEdges,pathNodeInversionRate

In [ ]:
#| export
def getBlockEdges(graph = None,
                   nodeLengths=None, pathLengths = None, 
                   pathNodeArray = None, pathDirArray = None,
                   pathNextNode = None, forwardLinks = None,
                   inversionThreshold=0.5):
    # Preparing paths for link-block identifications
    
    if nodeLengths is None or pathLengths is None or pathNodeArray is None or pathDirArray is None and pathNextNode is not None or forwardLinks is None:
        if graph is not None:
            forwardLinks = graph.forwardLinks
            
            nodeLengths = calcNodeLengths(graph)

            pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum = initialPathAnalysis(graph,nodeLengths)
        else:
            raise AttributeError(f'Either `graph` or `nodeLengths`, `pathLengths`, `pathNodeArray` and `pathDirArray` should be provided.')

    pathNodeInversionRate,nodesStructure,combinedNodeDirArray = getNodesStructurePathNodeInversionRate(pathNodeArray,pathDirArray,pathLengths,inversionThreshold=inversionThreshold)
    
    blockEdges = identifyRearrangementBlocks(nodesStructure,nodeLengths)
    
    return blockEdges,pathNodeInversionRate

## Generating zoom layer

This set of functions (with `nextLayerZoom` being main orchestration function) doing the job of generating next zoom level by collapsing
columns and then components together after smaller non-linear links are removed (by different set of functions).

### Finalising bin and component

In [ ]:
#| export
def addLink(fromComp,fromStrand,toComp,toStrand,pathList,fromComponentLinks,toComponentLinks):
    fromComponentLinks.setdefault(fromComp,{}).setdefault(fromStrand,{}).\
                       setdefault(toComp,{}).setdefault(toStrand,set()).\
                       update(pathList)
    toComponentLinks.setdefault(toComp,{}).setdefault(toStrand,{}).\
                     setdefault(fromComp,{}).setdefault(fromStrand,set()).\
                     update(pathList)

In [ ]:

def getOccInvChange(binColLengths,binBlockLength,binOcc,binInv,prevOcc,prevInv,inversionThreshold=0.5):
    occChanged = False
    invChanged = False
    occ = {}
    inv = {}
    
    for pathID in binOcc:
        
        # Averaging occupancy
        occ[pathID] = sum([bl*bo for bl,bo in zip(binColLengths,binOcc[pathID])])/binBlockLength
        # Do comparison through floor and then abs difference > 0
        if np.abs(np.floor(occ[pathID]+0.5)-np.floor(prevOcc.get(pathID,occ[pathID])+0.5))>0 \
            and occ[pathID]>0.5 and prevOcc.get(pathID,occ[pathID])>0.5:
            occChanged = True
        prevOcc[pathID] = occ[pathID]
        
        # Averaging invertion
        inv[pathID] = sum([bl*bo*bi for bl,bo,bi in zip(binColLengths,binOcc[pathID],binInv[pathID])])/(binBlockLength*occ[pathID])
        if (inv[pathID]-inversionThreshold)*(prevInv.get(pathID,inv[pathID])-inversionThreshold)<0 or \
        (inv[pathID]-inversionThreshold)*(prevInv.get(pathID,inv[pathID])-inversionThreshold)==0 and \ 
        inv[pathID]*prevInv.get(pathID,inv[pathID])>inversionThreshold*inversionThreshold:
            # The second comdition after `or` is taking the case where one is equal to inversionThreshold
            # and another is more than inversionThreshold.
            invChanged = True
        prevInv[pathID] = inv[pathID]
        
    return occChanged,invChanged,occ,inv,prevOcc,prevInv

In [ ]:
#| export
def getOccInv(binColLengths,binBlockLength,binOcc,binInv,inversionThreshold=0.5):
    occ = {}
    inv = {}
    
    for pathID in binOcc:
        # Averaging occupancy
        occ[pathID] = sum([bl*bo for bl,bo in zip(binColLengths,binOcc[pathID])])/binBlockLength
        
        # Averaging invertion
        inv[pathID] = sum([bl*bo*bi for bl,bo,bi in zip(binColLengths,binOcc[pathID],binInv[pathID])])/(binBlockLength*occ[pathID])

    return occ,inv

In [ ]:
#| export
def combineIntervals(posPath):
    posArray = np.array(posPath)
    posArray = posArray[np.argsort(posArray[:,0]),:]
    posIntersect = (posArray[1:,1]-(posArray[:-1,0]-1))*\
                    (posArray[:-1,1]-(posArray[1:,0]-1))
    newPos = [[posArray[0,0]]]
    candidates = [posArray[0,1]]
    for jointNum in range(len(posIntersect)):
        if posIntersect[jointNum]>=0:
            candidates.extend(posArray[jointNum+1,:].tolist())
        else:
            newPos[-1].append(np.max(candidates))
            newPos.append([posArray[jointNum+1,0]])
            candidates = [posArray[jointNum+1,1]]

    newPos[-1].append(np.max(candidates))    
    return newPos

In [ ]:
#| export
def recordBinZoom(occ,inv,binPosArray,nBins,nCols,
                  binBlockLength,binBlockLengths,
                  binColLengths,
                  binColStart,binColStarts,binColEnd,binColEnds,
                  matrix,inversionThreshold=0.5):
    # need to check for occupancy and inversion change in comparison with previous bin
    # and if that happens, break the component before this bin (should be special boolean returned.
    
    for pathID in binPosArray:
        pathMatrix = matrix.setdefault(pathID,[[],[]])
        pathMatrix[0].append(nBins)
        
        # Adding everything to matrix element with Combined positions and Annotations (already combined through set)
        pathMatrix[1].append([occ[pathID],inv[pathID],combineIntervals(binPosArray[pathID])])
    
    binBlockLengths.append(binBlockLength)
    
    binColStarts.append(binColStart)
    binColEnds.append(binColEnd)
    
    return 0,[],binColStarts,binColEnds,{},{},{},nBins+1,nCols+binBlockLength,binBlockLengths,matrix
    # binBlockLength,binColLengths,binColStarts,binColEnds,
    # binOcc,binInv,binPosArray,binAnn,nBins,nCols,
    # binBlockLengths,matrix

In [ ]:
#| export
def getAverageInv(binBlockLengths,matrixPathArray):
    num = 0
    dem = 0
    for blockCols,matrixEl in zip(binBlockLengths,matrixPathArray[1]):
        num += blockCols*matrixEl[1]
        dem += blockCols
    return num/dem

In [ ]:
#| export
def finaliseComponentZoom(component,components,componentLengths,#componentNucleotides,
                          nBins,nCols,occupants,
                          binBlockLengths,binColStarts,binColEnds,
                          matrix,starts,ends,
                          forwardPaths,invertedPaths,
                          compInvNum,compInvDen,inversionThreshold=0.5):

    componentLengths.append(nBins)
    component["matrix"].extend(sorted([[pathID,
                                 (compInv := int(compInvNum[pathID]/compInvDen[pathID]>inversionThreshold)),
                                 [matrixPathArray[0],matrixPathArray[1][::(-1 if compInv else 1)]]] \
                                for pathID,matrixPathArray in matrix.items()],key=lambda el: el[0]))
    
    component['binsToCols'] = binBlockLengths
    component["occupants"] = sorted(list(matrix.keys()))
    
    component['last_bin'] = component['first_bin'] + nBins - 1
    
    component['firstCol'] = min(binColStarts)
    component['lastCol'] = max(binColEnds)
    
    # Is it needed?
    component['binColStarts'] = binColStarts
    component['binColEnds'] = binColEnds
    
    curStarts = starts.intersection(forwardPaths)
    if len(curStarts)>0:
        component['starts'] = list(curStarts)
    starts -= set(forwardPaths)
        
    curEnds = ends.intersection(invertedPaths)
    if len(curEnds)>0:
        component['ends'] = list(curEnds)
    ends -= set(invertedPaths)    
        
    firstBin = component['last_bin'] + 1
    components.append(component)
    component = deepcopy(componentTemplate)
    component['first_bin'] = firstBin
    return component,components,componentLengths,0,0,set(),[],[],[],{},starts,ends,{},{}
    # component,components,componentLengths,(componentNucleotides),nBins,nCols,occupants,
    # binBlockLengths,binColStarts,binColEnds,matrix,starts,ends,compInvNum,compInvDen

In [ ]:
#| export
def finaliseBinZoom(compNum,
                    binOcc,binInv,binPosArray,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    newComponent,newComponents,newComponentLengths,#compAccDir,#newComponentNucleotides,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,linkLengths,
                    starts,ends,
                    forwardPaths,invertedPaths,pathsToInversion,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=0.5):
    
    # Consider removing this condition and joining all bins together. But not now :)
    occ,inv = getOccInv(binColLengths,binBlockLength,binOcc,binInv,inversionThreshold=inversionThreshold)
    
    for pathID in pathsToInversion:
        invertedPaths.add(pathID)
        forwardPaths -= set([pathID])

    
    binBlockLength,binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,nBins,nCols,binBlockLengths,matrix = \
    recordBinZoom(occ,inv,binPosArray,
                    nBins,nCols,binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    inversionThreshold=inversionThreshold)
    
    return binColLengths,binColStarts,binColEnds,\
            binOcc,binInv,binPosArray,nBins,nCols,\
            binBlockLength,binBlockLengths,matrix,\
            newComponent,newComponents,newComponentLengths,\
            newFromComponentLinks,newToComponentLinks,\
            occupants,\
            starts,ends,newToOldInd,oldToNewInd,\
            forwardPaths,invertedPaths,set() # The last one is pathsToInversion

### Break component?

In [ ]:
#| export
def getMatrixPathElement(matrix,pathID):
    res = [el for el in matrix if el[0]==pathID]
    if len(res)==1:
        return res[0]
    elif len(res)>0:
        warnings.warn(f"More than one element for path {pathID} is found!")
    
    return None

In [ ]:
#| export
def checkChange(compNum,components,zoomLevel,blockEdges):
    doBreak = False
    curComp = components[compNum]
    
    if compNum+1 in blockEdges:
        return True
    
    return doBreak

In [ ]:
#| export
def joinComponents(leftComp,rightComp, maxLengthComponent, inversionThreshold=0.5):
    '''
    !!! ⚠️ Currently not used
    
    If the joining was successful, the function will return a joined component.
    
    If the joining was not successful and was aborted for one of the following reasons, it will return a list of original components. 
    The reasons for aborting the joining can be the following:
    - In one of the paths the invertion is lower than threshold in one component and higher in the other.
    - Left component contains at least one end
    - Right component contains at least one start
    
    The function will not check links for coming or going on the right of the left component and left of the right component. 
    It will just get left links from left component and right links from right component and assign them to the new component.
    '''
    
    if leftComp['last_bin']-leftComp['first_bin']+1 + rightComp['last_bin']-rightComp['first_bin']+1 > maxLengthComponent:
        return [leftComp,rightComp]
    
    if leftComp.get('ends',False):
        # End of a path
        return [leftComp,rightComp]
    
    newComp = {}
    newComp['first_bin'] = min(leftComp['first_bin'],rightComp['first_bin'])
    newComp['last_bin'] = max(leftComp['last_bin'],rightComp['last_bin'])
    newComp['firstCol'] = min(leftComp['firstCol'],rightComp['firstCol'])
    newComp['lastCol'] = max(leftComp['lastCol'],rightComp['lastCol'])
    
    leftCompNumBins = leftComp['last_bin']-leftComp['first_bin']+1
    
    newComp['occupants'] = list(set(leftComp['occupants']).union(rightComp['occupants']))
    
    for pathID in newComp['occupants']:
        leftPathElement = getMatrixPathElement(leftComp['matrix'],pathID)
        rightPathElement = getMatrixPathElement(rightComp['matrix'],pathID)
        if leftPathElement is None and rightPathElement is None:
            continue
        
        if leftPathElement is None:
            if len([el for el in rightPathElement[2][1] if el[2][0][0]==1 or el[2][-1][0]==1])>0:
                # Start of a path
                return [leftComp,rightComp]
            rightPathElement[2][0] = [el+leftCompNumBins for el in rightPathElement[2][0]]
            newComp.setdefault("matrix",[]).append(rightPathElement)
            continue
        
        if rightPathElement is None:
            newComp.setdefault("matrix",[]).append(leftPathElement)
            continue
        
        if (leftPathElement[1]>inversionThreshold and rightPathElement[1]<=inversionThreshold) or \
           (leftPathElement[1]<=inversionThreshold and rightPathElement[1]>inversionThreshold):
            return [leftComp,rightComp]
        
        newPathElement = []
        newPathElement.append(pathID)
        newPathElement.append(leftPathElement[1])
        pathMatrix = []
        pathMatrix.append(leftPathElement[2][0] + [el+leftCompNumBins for el in rightPathElement[2][0]])
        pathMatrix.append(leftPathElement[2][1] + rightPathElement[2][1])
        newPathElement.append(pathMatrix)
        newComp.setdefault("matrix",[]).append(newPathElement)
        
    newComp['larrivals'] = leftComp['larrivals']
    newComp['ldepartures'] = leftComp['ldepartures']
    newComp['rarrivals'] = rightComp['rarrivals']
    newComp['rdepartures'] = rightComp['rdepartures']
    ends = list(set(leftComp.get('ends',[])).union(rightComp.get('ends',[])))
    if len(ends)>0:
        newComp['ends'] = ends
    
    return newComp

In [ ]:
#| export
def checkLinksZoom(compNum,fromComponentLinks,toComponentLinks):
    # Check only outgoing from the right and incoming to the right
    # and check outgoing from the left on and incoming to the left on the next node.
    if any(['-' in toNodeDicts for toNodeDicts in fromComponentLinks.get(compNum+1,{}).get('+',{}).values()]):
        # If there is an outgoing link from positive block (right) to negative block (anywhere)
        return True

    if any(['+' in fromNodeDicts for fromNodeDicts in toComponentLinks.get(compNum+1,{}).get('-',{}).values()]):
        # If there is an incoming link to the negative block (right) from positive block (anywhere)
        return True
    
    if any([toNodes!=compNum+2 for toNodes in fromComponentLinks.get(compNum+1,{}).get('+',{}).keys()]):
        # If there is an outgoing link from positive block (right) to anywhere 
        # (positive block because negative downstream was excluded above) except the next component
        return True
    
    if any([fromNodes!=compNum+2 for fromNodes in toComponentLinks.get(compNum+1,{}).get('-',{}).keys()]):
        # If there is an incoming link to negative block (right) from anywhere
        # (negavive block because positive upstream was excluded above) except the next component
        return True
    
    if any([fromNodes!=compNum+1 or '-' in fromNodesDict for fromNodes,fromNodesDict in toComponentLinks.get(compNum+2,{}).get('+',{}).items()]):
        # If there is an incoming link to positive block of next component (left) from anywhere except current component
        return True
    
    if any([toNodes!=compNum+1 or '+' in toNodesDict for toNodes,toNodesDict in fromComponentLinks.get(compNum+2,{}).get('-',{}).items()]):
        # If there is an outgoing link from negative block (left) to anywhere except current component
        return True
    
    if not (('+' in fromComponentLinks.get(compNum+1,{}).get('+',{}).get(compNum+2,{})) or \
            ('-' in toComponentLinks.get(compNum+1,{}).get('-',{}).get(compNum+2,{}))):
        return True
    
    return False

In [ ]:
#| export
def checkForBreaksZoom(zoomLevel,compNum,components,fromComponentLinks,toComponentLinks, blockEdges):
    if compNum<len(components)-1:
        breakByLinks = checkLinksZoom(compNum,fromComponentLinks,toComponentLinks)
        breakByChange = checkChange(compNum,components,zoomLevel, blockEdges)
    else:
        return [false]
    return [breakByLinks,breakByChange]

### Update links

In [ ]:
#| export
def splitPositiveNegative(compID,accs,components):
    '''
    This function simply pulls all accession presented in the component and split them into forward and inversed.
    
    Parameters
    ==========
    
    `compID`: int. Number of the component in the current zoom layer (0-based).
    `accs`: int or Iterable. Should provide either overall number of accession/paths in the graph or 
            a list of all (intended) accessions for the given component. It is used only for carrying over links 
            through empty components for some accessions.
    `components`: list[dict]. List of component dictionaries, one of the main data structure representing zoom layer.
    
    Returns
    =======
    
    `posAcc`: list[int]. IDs of accession which has forward direction in given component.
    `negAcc`: list[int]. IDs of accession which has inverse direction in given component.
    
    '''
    if isinstance(accs,int):
        emptyAcc = set(range(accs))
    elif isinstance(accs,Iterable):
        emptyAcc = set(accs)
    else:
        raise TypeError(f'`accs` should be either int or Iterable, but {type(accs)} was given.')
    posAcc = []
    negAcc = []
    for pathID,pathInversion,_ in components[compID]['matrix']:
        emptyAcc -= set([pathID])
        if pathInversion==1:
            negAcc.append(pathID)
        else:
            posAcc.append(pathID)
    
    return posAcc,negAcc,emptyAcc

In [ ]:
#| export
def intersectAccLists(accList,dirDict):
    overallLinkAccList = set()
    for linkAccList in dirDict.values():
        overallLinkAccList.update(linkAccList)
    return set(accList).intersection(overallLinkAccList)

In [ ]:
#| export
def updateLinks(newToOldInd,oldToNewInd,
                fromComponentLinks,toComponentLinks,
                linkLengths,pairedLinks,interconnectedLinks,blockEdges,accStarts,accEnds,components,compAccDir,
                newFromComponentLinks={},newToComponentLinks={}):
    '''
    newToOldInd and oldToNewInd: both index and values are 0-based numbers of components 
    in previous and current zoomlayer.
    
    
    '''
    for newComp,oldCompList in enumerate(newToOldInd):
        leftOldCompId = oldCompList[0] + 1
        rightOldCompId = oldCompList[-1] + 1
        newCompId = newComp + 1
        
        newCompPosAcc,newCompNegAcc,newCompEmptyAcc = splitPositiveNegative(newComp,list(accStarts.keys()),components)
        
        # The next two if blocks set conditions for processing or not processing left and right of the current new component
        # The side should not be processed if the end of the previous node is coming from the same component 
        # (on lower zoom level) as the begininng of the current component.
        if newComp>0:
            if oldCompList[0]==newToOldInd[newComp-1][-1]:
                doLeft = False
            else:
                doLeft = True
        else:
            doLeft = True
            
        if newComp<len(newToOldInd)-1:
            if oldCompList[-1]==newToOldInd[newComp+1][0]:
                doRight = False
            else:
                doRight = True
        else:
            doRight = True
        
        # Departure on the right (from positive block)
        if doRight:
            fromComp = rightOldCompId
            for toCompDict in fromComponentLinks.get(fromComp,{}).values():
                for toOldComp,toOldCompDirDict in toCompDict.items():
                    # To positive strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][0]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[toNewCompId][0]==toOldComp-1:
                        # Getting accession list which come out from positive and goes to positive.
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(toCompPosAcc)),
                                                    toOldCompDirDict)

                        if len(accList)>0:
                            if not(toNewCompId+1==newCompId and \
                                (fromComp-1!=newToOldInd[toNewCompId][-1] or toOldComp-1!=newToOldInd[toNewCompId][0])):
                                # If the link is self loop (which happens sometimes), then it has to come out from right component
                                # and go to the left component, otherwise it should not be carried forward.
                                addLink(newCompId,'+',toNewCompId+1,'+',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)
                        
                        ### Wrap creating empty links into a separate function!!!
                        if fromComp+1==toOldComp and newCompId+1==toNewCompId+1:
                            continuityLinks = set(fromComponentLinks.get(fromComp,{}).get('+',{}).get(fromComp+1,{}).get('+',[]))
                            emptyLinks = (set(newCompPosAcc)|set(newCompEmptyAcc)).intersection(set(toCompPosAcc)|set(toCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(newCompId,'+',toNewCompId+1,'+',emptyLinks,newFromComponentLinks,newToComponentLinks)

                    # To negative strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][-1]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the right of the new comp
                    if newToOldInd[toNewCompId][-1]==toOldComp-1:
                        # Getting accession list which come out from positive and goes to negative.
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(toCompNegAcc)),
                                                    toOldCompDirDict)
                        if len(accList)>0:
                            addLink(newCompId,'+',toNewCompId+1,'-',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)
        
        # Departure on the left (from negative block)
        if doLeft:
            fromComp = leftOldCompId
            for toCompDict in fromComponentLinks.get(fromComp,{}).values():
                for toOldComp,toOldCompDirDict in toCompDict.items():
                    # To positive strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][0]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[toNewCompId][0]==toOldComp-1:
                        # Getting accession list which come out from negative and goes to positive.
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(toCompPosAcc)),
                                                    toOldCompDirDict)
                        if len(accList)>0:
                            addLink(newCompId,'-',toNewCompId+1,'+',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)

                    # to Negative strand
                    # Getting to comp (new) id
                    toNewCompId = oldToNewInd[toOldComp-1][-1]
                    toCompPosAcc,toCompNegAcc,toCompEmptyAcc = splitPositiveNegative(toNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the right of the new comp
                    if newToOldInd[toNewCompId][-1]==toOldComp-1:
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(toCompNegAcc)),
                                                    toOldCompDirDict)
                        if len(accList)>0:
                            if not (toNewCompId+1==newCompId and \
                                (fromComp-1!=newToOldInd[toNewCompId][0] or toOldComp-1!=newToOldInd[toNewCompId][-1])):
                                # If the link is self loop (which happens sometimes), then it has to come out from left component
                                # and go to the right component, otherwise it should not be carried forward.
                                addLink(newCompId,'-',toNewCompId+1,'-',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)

                        if fromComp-1==toOldComp and newCompId-1==toNewCompId+1:
                            continuityLinks = set(fromComponentLinks.get(fromComp,{}).get('-',{}).get(fromComp+1,{}).get('-',[]))
                            emptyLinks = (set(newCompNegAcc)|set(newCompEmptyAcc)).intersection(set(toCompNegAcc)|set(toCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(newCompId,'-',toNewCompId+1,'-',emptyLinks,newFromComponentLinks,newToComponentLinks)


        # Arrival on the left (to positive block)
        if doLeft:
            toComp = leftOldCompId
            for fromCompDict in toComponentLinks.get(toComp,{}).values():
                for fromOldComp,fromOldCompDirDict in fromCompDict.items():
                    # From positive strand
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][-1]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the from comp (old) is on the right of the new comp
                    if newToOldInd[fromNewCompId][-1]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(fromCompPosAcc)),
                                                    fromOldCompDirDict)
                        if len(accList)>0:
                            if not (fromNewCompId+1==newCompId and \
                                (fromOldComp-1!=newToOldInd[fromNewCompId][-1] or toComp-1!=newToOldInd[fromNewCompId][0])):
                                # If the link is self loop (which happens sometimes), then it has to come out from right component
                                # and go to the left component, otherwise it should not be carried forward.
                                addLink(fromNewCompId+1,'+',newCompId,'+',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)
                        if toComp-1==fromOldComp and newCompId-1==fromNewCompId+1:
                            continuityLinks = set(toComponentLinks.get(toComp,{}).get('+',{}).get(toComp-1,{}).get('+',[]))
                            emptyLinks = (set(newCompPosAcc)|set(newCompEmptyAcc)).intersection(set(fromCompPosAcc)|set(fromCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(fromNewCompId+1,'+',newCompId,'+',emptyLinks,newFromComponentLinks,newToComponentLinks)

                    # From negative strand
                    # Getting to comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][0]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the to comp (old) is on the left of the new comp
                    if newToOldInd[fromNewCompId][0]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompPosAcc).intersection(fromCompNegAcc)),
                                                    fromOldCompDirDict)
                        if len(accList)>0:
                            addLink(fromNewCompId+1,'-',newCompId,'+',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)

        if doRight:
            # Arrival on the right (to negative block)
#             for toComp in [leftOldCompId,rightOldCompId]:
            toComp = rightOldCompId
            for fromCompDict in toComponentLinks.get(toComp,{}).values():
                for fromOldComp,fromOldCompDirDict in fromCompDict.items():
                    # From positive strand
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][-1]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the from comp (old) is on the right of the new comp
                    if newToOldInd[fromNewCompId][0]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(fromCompPosAcc)),
                                                    fromOldCompDirDict)
                        if len(accList)>0:
                            addLink(fromNewCompId+1,'+',newCompId,'-',
                                    accList,
                                    newFromComponentLinks,newToComponentLinks)

                    # From negative strand
                    # Getting from comp (new) id
                    fromNewCompId = oldToNewInd[fromOldComp-1][0]
                    fromCompPosAcc,fromCompNegAcc,fromCompEmptyAcc = splitPositiveNegative(fromNewCompId,list(accStarts.keys()),components)
                    # Check whether the from comp (old) is on the left of the new comp
                    if newToOldInd[fromNewCompId][-1]==fromOldComp-1:
                        accList = intersectAccLists(list(set(newCompNegAcc).intersection(fromCompNegAcc)),
                                                    fromOldCompDirDict)
                        if len(accList)>0:
                            if not (fromNewCompId+1==newCompId and \
                                (fromOldComp-1!=newToOldInd[fromNewCompId][0] or toComp-1!=newToOldInd[fromNewCompId][-1])):
                                # If the link is self loop (which happens sometimes), then it has to come out from left component
                                # and go to the right component, otherwise it should not be carried forward.
                                addLink(fromNewCompId+1,'-',newCompId,'-',
                                        accList,
                                        newFromComponentLinks,newToComponentLinks)

                        if toComp+1==fromOldComp and newCompId+1==fromNewCompId+1:
                            continuityLinks = set(toComponentLinks.get(toComp,{}).get('-',{}).get(toComp-1,{}).get('-',[]))
                            emptyLinks = (set(newCompNegAcc)|set(newCompEmptyAcc)).intersection(set(fromCompNegAcc)|set(fromCompEmptyAcc)).intersection(continuityLinks)
                            if len(emptyLinks)>0:
                                addLink(fromNewCompId+1,'-',newCompId,'-',emptyLinks,newFromComponentLinks,newToComponentLinks)
    
    for acc in accStarts.keys():
        accStarts[acc] = oldToNewInd[accStarts[acc]-1][0]+1
        accEnds[acc] = oldToNewInd[accEnds[acc]-1][0]+1
    
    linkLengthsConv,pairedLinksConv,interconnectedLinksConv,blockEdgesConv = \
        convertRemovableComponents(oldToNewInd,linkLengths,pairedLinks,interconnectedLinks,blockEdges,fromComponentLinks)
    return newFromComponentLinks,newToComponentLinks,accStarts,accEnds,linkLengthsConv,pairedLinksConv,interconnectedLinksConv,blockEdgesConv

### Main layer generation function + assistant function

In [ ]:
#| export
def isStartEnd(compNum,components):
    isBreak = False
    leftComp = components[compNum]
    if compNum<len(components)-1:
        rightComp = components[compNum+1]
    else:
        rightComp = None
    
    
    for path,compInvBin,pathMatrix in leftComp['matrix']:
        if compInvBin==1:
            if path in leftComp.get('starts',[]):
                isBreak = True
        else:
            if path in leftComp.get('ends',[]):
                isBreak = True
    
    if rightComp is not None:
        for path,compInvBin,pathMatrix in rightComp['matrix']:
            if compInvBin==1:
                if path in rightComp.get('ends',[]):
                    isBreak = True
            else:
                if path in rightComp.get('starts',[]):
                    isBreak = True
    
    return isBreak

In [ ]:
#| export
def nextLayerZoom(zoomLevel,components,componentLengths,#componentNucleotides,
                  fromComponentLinks,toComponentLinks,graph,
                  accStarts,accEnds,
                  maxLengthComponent,
                  linkLengths,pairedLinks,interconnectedLinks,blockEdges,
                  inversionThreshold=0.5,
                  debug=False,debugTime=False):

    print('\n===========================')
    print(f'Zoom level {zoomLevel}')
    print('===========================')
    
    numComponents = len(components)
    numComponentsDigits = int(np.ceil(np.log10(numComponents)))
    
    newComponent = deepcopy(componentTemplate)
    newComponents = []
    newComponentLengths = []
    newComponentNucleotides = []

    newFromComponentLinks = {}
    newToComponentLinks = {}
    

    occupants = set()
    newToOldInd = [[]]
    oldToNewInd = []
    
    binMeanOcc = {}
    binMeanInv = {}
    binOcc = {}
    binInv = {}
    binPosArray = {}
    binAnn = {}
    
    binColStarts = []
    binColEnds = []
    binColLengths = []
    binBlockLength = 0
    binBlockLengths = []

    matrix = {}
    nBins = 0
    nCols = 0

    starts = set()
    ends = set()

    compInvNum = {}
    compInvDen = {}
    
    compAccDir = {}
    
    for compNum,component in enumerate(components):
        if debug or debugTime:
            print(f'Processing component {compNum+1:0{numComponentsDigits}}/{numComponents:0{numComponentsDigits}}')
        else:
            print(f'\rProcessing component {compNum+1:0{numComponentsDigits}}/{numComponents:0{numComponentsDigits}}',end='')
        pathsToInversion = set()
        forwardPaths = set(range(len(graph.accessions)))
        invertedPaths = set()
        
        oldToNewInd.append([])
        newToOldInd[-1].append(compNum)
        occupants.update(component['occupants'])
        starts.update(component.get('starts',[]))
        ends.update(component.get('ends',[]))
        compNumBins = component['last_bin']-component['first_bin']+1
        for binNum in range(0,compNumBins):
            if binBlockLength>0:
                binColStart = min(binColStart,component['binColStarts'][binNum])
                binColEnd = max(binColEnd,component['binColEnds'][binNum])
            else:
                binColStart = component['binColStarts'][binNum]
                binColEnd = component['binColEnds'][binNum]

            binColLengths.append(component['binsToCols'][binNum])
            binBlockLength += binColLengths[-1]
            # check this process for order of added bins
            # compInvBin should be calculated on the fly as average of added bins weighted by colLength
            # It should be kept until the component closed.
            # If average inversion changes from one side to the other, then inverted paths should be changed,
            # and possibly some lists should be reversed.
            
            for pathID,oldCompInv,pathMatrix in component['matrix']:
                compInvBin = compInvNum.get(pathID,oldCompInv)/compInvDen.get(pathID,1)
                if compInvBin==1:
                    pathsToInversion.add(pathID)
                    compAccDir.setdefault(compNum,{})[pathID]='-'
                else:
                    pathsToInversion.discard(pathID)
                    compAccDir.setdefault(compNum,{})[pathID]='+'
                occupiedBins = pathMatrix[0]
                binsMatrix = pathMatrix[1]

                try:
                    # For inverted old components reading should happen from the other side to the front.
                    if oldCompInv==1:
                        binPos = occupiedBins.index(componentLengths[compNum] - 1 - binNum)
                    else:
                        binPos = occupiedBins.index(binNum)
                        
                    binOcc.setdefault(pathID,[]).append(binsMatrix[binPos][0])
                    binInv.setdefault(pathID,[]).append(binsMatrix[binPos][1])
                    binPosArray.setdefault(pathID,[]).extend(binsMatrix[binPos][2])
                    
                    compInvNum[pathID] = compInvNum.get(pathID,0) + binColLengths[-1]*binOcc[pathID][-1]*binInv[pathID][-1]
                    compInvDen[pathID] = compInvDen.get(pathID,0) + binColLengths[-1]*binOcc[pathID][-1]
                except ValueError:
                    continue
            
            # Finding out the size of the next bin to add
            if binNum+1<compNumBins:
                #If this component (old) has not ended yet, then get the size of the next bin
                nextBinColLength = component['binsToCols'][binNum+1]
            elif compNum+1<len(components):
                # If we already read the last bin of the current component
                # and it is not the last component, then we can do one of the two options:
                
                # Option 1: Get the first bin of the next component
                # This will allow us to avoid adding a large bin to the small last bin of the previous component.
                # But if the first bin of the next component was really small (can happen only if it was a a one bin component 
                # or there was a particular reason to break bin, it will still allow to add part of the component to the newly 
                # forming bin and the rest to the new bin
                nextBinColLength = components[compNum+1]['binsToCols'][0]
                
                # Option 2: Get the whole size of the next component
                # This option will not allow spliting a component in any way in such a way that a bin will get pieces of two 
                # neighbouring components. It will allow adding a very small single bin component to the bin which partially 
                # used by the end of previous component.
                # nextBinColLength = sum(components[compNum+1]['binsToCols'])
                
            else:
                nextBinColLength = 0
            if binBlockLength+nextBinColLength > zoomLevel and binBlockLength>0:
                
                # If bin got equal or larger than target zoom level bin size 
                # (it can grow over by less than previous zoom level)
                # then bin is closing and new bin will be formed.
                [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,nBins,nCols,
                    binBlockLength,binBlockLengths,matrix,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,
                    starts,ends,newToOldInd,oldToNewInd,
                    forwardPaths,invertedPaths,pathsToInversion] = \
                finaliseBinZoom(compNum,binOcc,binInv,binPosArray,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,linkLengths,
                    starts,ends,
                    forwardPaths,invertedPaths,pathsToInversion,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=inversionThreshold)

            if nBins==maxLengthComponent:
                
                if nBins>0:
                
                    # Add links from current component to the next one
                    if len(forwardPaths)>0:
                        addLink(len(newComponents)+1,'+',len(newComponents)+2,'+',forwardPaths,newFromComponentLinks,newToComponentLinks)
                    if len(invertedPaths)>0:
                        addLink(len(newComponents)+2,'-',len(newComponents)+1,'-',invertedPaths,newFromComponentLinks,newToComponentLinks)
                    # Add new element to newToOldInd
                    
                    if binNum==componentLengths[compNum]-1:
                        newToOldInd.append([])
                    else:
                        newToOldInd.append([compNum])
                    # Add number of next element to oldToNewInd
                    oldToNewInd[-1].append(len(newComponents))
                    
                    # close component

                    newComponent,newComponents,newComponentLengths,\
                    nBins,nCols,occupants,binBlockLengths,binColStarts,binColEnds,\
                    matrix,starts,ends,compInvNum,compInvDen = \
                        finaliseComponentZoom(newComponent,newComponents,newComponentLengths,
                                              nBins,nCols,occupants,binBlockLengths,
                                              binColStarts,binColEnds,
                                              matrix,starts,ends,
                                              forwardPaths,invertedPaths,
                                              compInvNum,compInvDen,inversionThreshold=inversionThreshold)
                    occupants = set(component['occupants'])
        
        isEndBreak = isStartEnd(compNum,components)
        
        if isEndBreak and binBlockLength>0:
            [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,nBins,nCols,
            binBlockLength,binBlockLengths,matrix,
            newComponent,newComponents,newComponentLengths,
            newFromComponentLinks,newToComponentLinks,
            occupants,
            starts,ends,newToOldInd,oldToNewInd,
            forwardPaths,invertedPaths,pathsToInversion] = \
                finaliseBinZoom(compNum,binOcc,binInv,binPosArray,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,linkLengths,
                    starts,ends,
                    forwardPaths,invertedPaths,pathsToInversion,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=inversionThreshold)
            
        if compNum==len(components)-1:
            # Close the bin
            if binBlockLength>0:
                [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,nBins,nCols,
                binBlockLength,binBlockLengths,matrix,
                newComponent,newComponents,newComponentLengths,
                newFromComponentLinks,newToComponentLinks,
                occupants,
                starts,ends,newToOldInd,oldToNewInd,
                forwardPaths,invertedPaths,pathsToInversion] = \
                    finaliseBinZoom(compNum,binOcc,binInv,binPosArray,
                        nBins,nCols,
                        binBlockLength,binBlockLengths,binColLengths,
                        binColStart,binColStarts,binColEnd,binColEnds,
                        matrix,
                        newComponent,newComponents,newComponentLengths,
                        newFromComponentLinks,newToComponentLinks,
                        occupants,linkLengths,
                        starts,ends,
                        forwardPaths,invertedPaths,pathsToInversion,
                        newToOldInd,oldToNewInd,
                        inversionThreshold=inversionThreshold)
            
            # break the last component
            if nBins>0:
                
                oldToNewInd[-1].append(len(newComponents))
                
                newComponent,newComponents,newComponentLengths,\
                nBins,nCols,occupants,binBlockLengths,binColStarts,binColEnds,\
                matrix,starts,ends,compInvNum,compInvDen = \
                    finaliseComponentZoom(newComponent,newComponents,newComponentLengths,
                                          nBins,nCols,occupants,binBlockLengths,
                                          binColStarts,binColEnds,
                                          matrix,starts,ends,
                                          range(len(graph.accessions)),range(len(graph.accessions)),
                                          compInvNum,compInvDen,inversionThreshold=inversionThreshold)

        elif isEndBreak or any(checkForBreaksZoom(zoomLevel,compNum,components,fromComponentLinks,toComponentLinks,blockEdges)):
            # Break at the end of the component because of the links
            if binBlockLength>0:
                [binColLengths,binColStarts,binColEnds,binOcc,binInv,binPosArray,nBins,nCols,
                binBlockLength,binBlockLengths,matrix,
                newComponent,newComponents,newComponentLengths,
                newFromComponentLinks,newToComponentLinks,
                occupants,
                starts,ends,newToOldInd,oldToNewInd,
                forwardPaths,invertedPaths,pathsToInversion] = \
                finaliseBinZoom(compNum,binOcc,binInv,binPosArray,
                    nBins,nCols,
                    binBlockLength,binBlockLengths,binColLengths,
                    binColStart,binColStarts,binColEnd,binColEnds,
                    matrix,
                    newComponent,newComponents,newComponentLengths,
                    newFromComponentLinks,newToComponentLinks,
                    occupants,linkLengths,
                    starts,ends,
                    forwardPaths,invertedPaths,pathsToInversion,
                    newToOldInd,oldToNewInd,
                    inversionThreshold=inversionThreshold)
            
            # Break component
            if nBins>0:
                
                newToOldInd.append([])
                oldToNewInd[-1].append(len(newComponents))
                
                newComponent,newComponents,newComponentLengths,\
                nBins,nCols,occupants,binBlockLengths,binColStarts,binColEnds,\
                matrix,starts,ends,compInvNum,compInvDen = \
                    finaliseComponentZoom(newComponent,newComponents,newComponentLengths,
                                          nBins,nCols,occupants,binBlockLengths,
                                          binColStarts,binColEnds,
                                          matrix,starts,ends,
                                          range(len(graph.accessions)),range(len(graph.accessions)),
                                          compInvNum,compInvDen,inversionThreshold=inversionThreshold)
                
        elif nBins>0 or binBlockLength>0:
            oldToNewInd[-1].append(len(newComponents))
    
    print('\nProcessing component finished.')
    
    newFromComponentLinks,newToComponentLinks,accStarts,accEnds,newLinkLengths,newPairedLinks,newInterconnectedLinks,newBlockEdges = \
        updateLinks(newToOldInd,oldToNewInd,fromComponentLinks,toComponentLinks,
                    linkLengths,pairedLinks,interconnectedLinks,blockEdges,
                    accStarts,accEnds,
                    newComponents,compAccDir,newFromComponentLinks,newToComponentLinks)
    
    print()
    
    return newComponents,newComponentLengths,\
            newFromComponentLinks,newToComponentLinks,\
            accStarts,accEnds,newLinkLengths,newPairedLinks,newInterconnectedLinks,newBlockEdges\
            ,oldToNewInd,newToOldInd

## Clear elements too small to show

This set of functions (with the orchestrating function being `clearInvisible`) look at earlier identified non-linear link to size (or number of nucleotides) 
associations and if the next zoom level is larger than some sizes, then these links are removed (with reinstating of some of linear links instead).

After that Isolation blocks are identified and removed. Isolation block is a contiguous block of components (columns) that are connected only to each other but not to any of components outside the block. 

### Removing links and rearrangement blocks associated to too small blocks

In [ ]:
#| export
def removeLink(fromComponentLinks,toComponentLinks,linkList,remLinks,link,pairedLink=None,subLink=None,subLinks=None,remLinkAccessions=None):
    '''
    This function remove the main link.
    
    If paired and substitute links are provided, the paired link will be checked (if it is not removed or 
    in the queue to be removed), it will be added to the queue
    
    After that common accessions for the same strand (for each separately) for start of main link and and end of paired link
    are found and substitute link is established for all such accessions.
    
    If the substitute link is not (k,k+1), but (k,k+p), then in componentLinks all links (k,k+1),(k+1,k+1),...,(k+p-1,k+p)
    are established.
    
    '''
    
    accessionMainLink = {}

    if pairedLink is not None and subLink is not None and subLinks is not None and remLinkAccessions is not None:
        processPair = True
    else:
        processPair = False
        
    if link not in remLinks: # Check validity.
        # removing links from fromComponentLinks
        for fromStrand,fromStrandDict in list(fromComponentLinks.get(link[0],{}).items()):

            try:
                if processPair:
                    toCompDict = fromComponentLinks[link[0]][fromStrand].pop(link[1])
                    for toStrand,accessionList in list(toCompDict.items()):
                        if toStrand==fromStrand and link[0]+(1 if toStrand=='+' else -1)==link[1]:
                            fromComponentLinks[link[0]][fromStrand].setdefault(link[1],{}).setdefault(toStrand,set()).update(accessionList)
                        accessionMainLink.setdefault(fromStrand,set()).update(accessionList)
                else:
                    allCleared = True
                    for toStrand,accessionList in list(fromStrandDict.get(link[1],{}).items()):
                        if toStrand!=fromStrand or link[0]+(1 if toStrand=='+' else -1)!=link[1]:
                            del fromComponentLinks[link[0]][fromStrand][link[1]][toStrand]
                        else:
                            allCleared = False
                    
                    if allCleared:
                        del fromComponentLinks[link[0]][fromStrand][link[1]]
            except KeyError:
                pass


            if len(fromComponentLinks[link[0]][fromStrand])==0:
                del fromComponentLinks[link[0]][fromStrand]

        if link[0] in fromComponentLinks:
            if len(fromComponentLinks[link[0]])==0:
                del fromComponentLinks[link[0]]

        # removing links from toComponentLinks
        for toStrand,toStrandDict in list(toComponentLinks.get(link[1],{}).items()):

            try:
                allCleared = True
                for fromStrand,accessionList in list(toStrandDict.get(link[0],{}).items()):
                    if toStrand!=fromStrand or link[0]+(1 if toStrand=='+' else -1)!=link[1]:
                        del toComponentLinks[link[1]][toStrand][link[0]][fromStrand]
                    else:
                        allCleared = False
                
                if allCleared:
                    del toComponentLinks[link[1]][toStrand][link[0]]
            except KeyError:
                pass

            if len(toComponentLinks[link[1]][toStrand])==0:
                del toComponentLinks[link[1]][toStrand]

        if link[1] in toComponentLinks:
            if len(toComponentLinks[link[1]])==0:
                del toComponentLinks[link[1]]

        remLinks.add(link)
    
    if processPair:
        if len(accessionMainLink)>0:
            remLinkAccessions[link]==accessionMainLink
        else:
            accessionMainLink = remLinkAccessions.get(link,{})
            
    if processPair and len(accessionMainLink)>0 and subLink not in subLinks:
        if pairedLink not in linkList and pairedLink not in remLinks:
            # If for some reason paired link is not in the list to be removed or removed already, 
            # it should also be removed. 
            # BE CAREFUL ❗❗❗
            # Although, if there is an issue with moving from layer to layer this can mask the issue.
            
            # The paired link is not removed here, it is added to the linkList and will be processed separately.
            linkList.add(pairedLink)
        
        # Here we need to get accession list for each toStrand for paired link (for the end of link)
        accessionPaired = {}
        for toStrand,toStrandDict in list(toComponentLinks.get(link[1],{}).items()):

            try:
                fromCompDict = toComponentLinks[link[1]][toStrand].pop(link[0])
                for fromStrand,accessionList in fromCompDict:
                    accessionPaired.setdefault(toStrand,set()).update(accessionList)
            except KeyError:
                pass
        
        subLinks.add(subLink)
        
        for strand,strandAccMain in accessionMainLink.items():
            # Then intersect it with accessionsMainLink
            strandAccPaired = accessionPaired.get(strand,set())
            accIntersect = strandAccMain & strandAccPaired

            # Establish substitute links 
            step = 1 if strand=='+' else -1
            
            for fromComp in range(subLink[0],subLink[1],step):
                toComp = fromComp + step
                if (strand=='+' and fromComp>toComp) or (strand=='-' and fromComp<toComp):
                    _fromComp = toComp
                    _toComp = fromComp
                else:
                    _fromComp = fromComp
                    _toComp = toComp
                    
                addLink(_fromComp,strand,_toComp,strand,accIntersect,fromComponentLinks,toComponentLinks)
        

In [ ]:
#| export
def processCollapsibleBlocks(zoomLevel,linkLengths,pairedLinks,interconnectedLinks,fromComponentLinks,toComponentLinks):
    
    # ❗Need to remove the links below zoom level everywhere, but the substitute links,
    # ❗associated with the pair should be established ONLY for paths where both links from the pair present!
    remLinkAccessions = {}
    for blockLength in sorted(list(linkLengths.keys())):
        if blockLength<zoomLevel:
            linkList = linkLengths.pop(blockLength)
            remLinks = set()
            subLinks = set()
            
            while len(linkList)>0:
                link = linkList.pop()
                try:
                    pairedDict = pairedLinks.pop(link)
                
                    for pairedLink,subLink in pairedDict.items():
                        removeLink(fromComponentLinks,toComponentLinks,linkList,remLinks,link,pairedLink,subLink,subLinks,remLinkAccessions)
                except KeyError:
                    removeLink(fromComponentLinks,toComponentLinks,linkList,remLinks,link)
                
                try:
                    interconnectedLinks.pop(link)
                except KeyError:
                    warnings.warn(f'Link {link} was not found in interconnectedLinks. Possibly it was removed earlier, but it is better to double check.')
        else:
            break

In [ ]:
#| export
def clearRearrangementBlocks(zoomLevel,blockEdges):
    return {edge:size for edge,size in blockEdges.items() if size>=zoomLevel}  

### Find isolated blocks

#### Identify empty edges

In [ ]:
#| export
def testStartEnd(compNum,isLeft,components,accStarts,accEnds):
    if isLeft:
        startInv = 0
        endInv = 1
    else:
        startInv = 1
        endInv = 0
        
    for accStart,startComp in accStarts.items():
        if startComp == compNum:
            comp = components[compNum-1]
            try:
                accessionInd = comp['occupants'].index(accStart)
            except ValueError:
                continue
            if comp['matrix'][accessionInd][1]==startInv:
                return True
    
    for accEnd,endComp in accEnds.items():
        if endComp == compNum:
            comp = components[compNum-1]
            try:
                accessionInd = comp['occupants'].index(accStart)
            except ValueError:
                continue
            if comp['matrix'][accessionInd][1]==endInv:
                return True
        
    return False

In [ ]:
#| export
def findEmptyEdges(fromComponentLinks,toComponentLinks,accStarts,accEnds,components):
    '''
    Identify all empty edges by simply finding components that do not appear either in toComponentLinks (left empty)
    or fromComponentLinks (right empty)
    '''
    
    allCompSet = set((np.array(range(len(components)))+1).tolist())
    toLinkedSet = set(toComponentLinks.keys())
    fromLinkedSet = set(fromComponentLinks.keys())

    leftEmptyCandidates = list(allCompSet - toLinkedSet)

    leftEmptyList = []

    for leftEmptyCand in leftEmptyCandidates:
        if testStartEnd(leftEmptyCand,True,components,accStarts,accEnds):
            continue

        if '-' in fromComponentLinks.get(leftEmptyCand,{}):
            continue

        leftEmptyList.append(leftEmptyCand)

    rightEmptyCandidates = list(allCompSet - fromLinkedSet)

    rightEmptyList = []

    for rightEmptyCand in rightEmptyCandidates:
        if testStartEnd(rightEmptyCand,False,components,accStarts,accEnds):
            continue

        if '-' in toComponentLinks.get(rightEmptyCand,{}):
            continue

        rightEmptyList.append(rightEmptyCand)
        
    return leftEmptyList,rightEmptyList

#### Identify isolated blocks

In [ ]:
#| export
def checkExternalLinks(blockStart,blockEnd,fromComponentLinks,toComponentLinks,components):
    '''
    
    
    Return
    ======
    
    componentID: int. If an external links found, then a list of components inside the block involved 
                    in external links returned. If no external links found, then empty list is returned.
    '''
    
    externalLinksComponents = []
    
    for compID in range(blockStart,blockEnd+1):
        passToNext = False
        
        if len(components[compID-1].get('ends',[]))>0 or len(components[compID-1].get('starts',[]))>0:
            externalLinksComponents.append(compID)
            continue
        
        for fromStrandDict in fromComponentLinks.get(compID,{}).values():
            for toComp in fromStrandDict.keys():
                if toComp>blockEnd or toComp<blockStart:
                    externalLinksComponents.append(compID)
                    passToNext = True
                    break
            if passToNext:
                break
        
        if passToNext:
            continue
            
        for toStrandDict in toComponentLinks.get(compID,{}).values():
            for fromComp in toStrandDict.keys():
                if fromComp>blockEnd or fromComp<blockStart:
                    externalLinksComponents.append(compID)
                    passToNext = True
                    break
            if passToNext:
                break
    
    return externalLinksComponents

In [ ]:
#| export
def createNewBoundaries(blockStart,blockEnd,externalLinksComps,leftEmptyList,rightEmptyList):
    leftEmptyArray = np.sort(np.array(leftEmptyList))
    rightEmptyArray = np.sort(np.array(rightEmptyList))
    
    if externalLinksComps[0]==blockStart:
        suitableStarts = np.where((leftEmptyArray>blockStart)&(leftEmptyArray<=blockEnd))[0]
        if len(suitableStarts)>0:
            blockSplit = [[leftEmptyArray[suitableStarts[0]]]]
        else:
            return []
    else:
        blockSplit = [[blockStart]]
    
    doEnd = True
    
    for breakNum,breakPoint in enumerate(externalLinksComps):
        
        # Adding end
        if doEnd:
            suitableEnds = np.where((rightEmptyArray>=blockSplit[-1][0]) & (rightEmptyArray<breakPoint))[0]
            if len(suitableEnds)>0:
                blockSplit[-1].append(rightEmptyArray[suitableEnds[-1]])
            else:
                del blockSplit[-1]
        
        #adding start
        if breakNum+1<len(externalLinksComps):
            nextBoundary = externalLinksComps[breakNum+1]
        else:
            nextBoundary = blockEnd+1
            
        if breakPoint<blockEnd:
            suitableStarts = np.where((leftEmptyArray>breakPoint) & (leftEmptyArray<=nextBoundary))[0]
            if len(suitableStarts)>0:
                blockSplit.append([leftEmptyArray[suitableStarts[0]]])
                doEnd = True
            else:
                doEnd = False
        else:
            doEnd = False
            break
    
    if doEnd:
        suitableEnds = np.where((rightEmptyArray>=blockSplit[-1][0]) & (rightEmptyArray<=blockEnd))[0]
        if len(suitableEnds)>0:
            blockSplit[-1].append(rightEmptyArray[suitableEnds[-1]])
        else:
            del blockSplit[-1]
    
    if len(blockSplit)>0:
        if len(blockSplit[0])>0:
            return blockSplit
        else:
            return []
    else:
        return []
    

In [ ]:
# Test for `createNewBoundaries`
import numpy as np

st = [2,5,6,8]
end = [2,3,4,6,8,9,10,11]

blocks = [[2,11],[2,3],[5,11],[8,11],[8,9],[8,11]]
blockSplits = [[[2,3],[5,11]],[[2,2]],[[6,6],[8,11]],[[8,9]],[[8,8]],[]]
externals = [[4],[3],[5,7],[10],[9],[8,9,10,11]]

for bl,blSpl,ext in zip(blocks,blockSplits,externals):
    blSplTT = createNewBoundaries(*bl,ext,st,end)
    assert blSpl == blSplTT,f'Expected {blSpl}, but got {blSplTT}'

In [ ]:
# Another test for `createNewBoundaries`
leftEmptyList = [2056, 3080, 3081, 2092, 2099, 1593, 3643, 2627, 1116, 2653, 2655, 3168, 2658, 613, 1637, 1638, 106, 1654, 2695, 2192, 1169, 1686, 2714, 3757, 2233, 3781, 723, 1240, 224, 1761, 1762, 1766, 3323, 1804, 786, 2331, 802, 2850, 807, 811, 1839, 1841, 3396, 3397, 1863, 3400, 843, 3423, 1898, 1899, 882, 884, 3463, 402, 2451, 3478, 408, 3482, 934, 426, 1962, 3504, 3516, 3519, 3520, 451, 1994, 1995, 972, 2506, 463, 3024, 1493, 1494, 3542, 1525]
rightEmptyList = [402, 2451, 3478, 407, 280, 3482, 2848, 802, 934, 807, 426, 811, 2091, 2092, 3757, 1839, 3504, 1841, 3516, 3519, 3405, 463, 722, 1240, 1761, 1762, 1766, 1899]
blockStart = 3396
blockEnd = 3405
externalLinksComps = [3396, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405]

createNewBoundaries(blockStart,blockEnd,externalLinksComps,leftEmptyList,rightEmptyList)

[]

In [ ]:
#| export
def identifyIsolatedBlocks(leftEmptyList,rightEmptyList,fromComponentLinks,toComponentLinks,components):
    if len(leftEmptyList)==0 or len(rightEmptyList)==0:
        return []
    
    leftmostLeftEmpty = min(leftEmptyList)
    rightmostRightEmpty = max(rightEmptyList)

    isolatedBlockCandidates = []
    isolatedBlockList = []

    if leftmostLeftEmpty<=rightmostRightEmpty:
        isolatedBlockCandidates.append([leftmostLeftEmpty,rightmostRightEmpty])

    while len(isolatedBlockCandidates)>0:

        isolatedBlock = isolatedBlockCandidates.pop()

        externalLinksComps = checkExternalLinks(*isolatedBlock,fromComponentLinks,toComponentLinks,components)

        if len(externalLinksComps)==0:
            isolatedBlockList.append(isolatedBlock)
        else:
            isolatedBlockCandidates.extend(createNewBoundaries(*isolatedBlock,externalLinksComps,leftEmptyList,rightEmptyList))
    
    return isolatedBlockList

### Removing Isolated Blocks

In [ ]:
#| export
def updateLinksRemoveComp(oldToNewInd,fromComponentLinks,toComponentLinks,
                          linkLengths,pairedLinks,interconnectedLinks,blockEdges,accStarts,accEnds):
    
    newFromComponentLinks={}
    newToComponentLinks={}
    
    for oldFromComp,fromCompDict in fromComponentLinks.items():
        if oldFromComp-1 in oldToNewInd:
            newFromCompDict = newFromComponentLinks.setdefault(oldToNewInd[oldFromComp-1][0]+1,{})
            for fromStrand,fromStrandDict in fromCompDict.items():
                newFromStrandDict = newFromCompDict.setdefault(fromStrand,{})
                for toComp,toCompDict in fromStrandDict.items():
                    if toComp-1 in oldToNewInd:
                        newFromStrandDict[oldToNewInd[toComp-1][0]+1] = toCompDict
                
    for oldToComp,toCompDict in toComponentLinks.items():
        if oldToComp-1 in oldToNewInd:
            newToCompDict = newToComponentLinks.setdefault(oldToNewInd[oldToComp-1][0]+1,{})
            for toStrand,toStrandDict in toCompDict.items():
                newToStrandDict = newToCompDict.setdefault(toStrand,{})
                for fromComp,fromCompDict in toStrandDict.items():
                    if fromComp-1 in oldToNewInd:
                        newToStrandDict[oldToNewInd[fromComp-1][0]+1] = fromCompDict
    
    for acc in accStarts.keys():
        accStarts[acc] = oldToNewInd[accStarts[acc]-1][0]+1
        accEnds[acc] = oldToNewInd[accEnds[acc]-1][0]+1
    
    newLinkLengths = {}
    for blockLength,linkList in linkLengths.items():
        for link in linkList:
            if (link[0]-1 not in oldToNewInd and link[1]-1 not in oldToNewInd):
                continue
            elif (link[0]-1 not in oldToNewInd or link[1]-1 not in oldToNewInd):
                raise RuntimeError(f'One side of link {link} is found in the isolated block, and another one not. \
                It is against definition of isolated block.')
            
            newLinkFrom = oldToNewInd[link[0]-1][0]+1
            newLinkTo = oldToNewInd[link[1]-1][0]+1
            newLinkLengths.setdefault(blockLength,set()).add((newLinkFrom,newLinkTo))
    
    newPairedLinks = {}
    for mainLink,pairedDict in pairedLinks.items():
        for pairedLink,subLink in pairedDict.items():
            isInIsolatedBlock = [side-1 not in oldToNewInd for side in itertools.chain(mainLink,pairedLink,subLink)]
            if all(isInIsolatedBlock):
                continue
            elif any(isInIsolatedBlock):
                raise RuntimeError(f'All sides of paired links {mainLink} - {pairedLink} : {subLink} should be \
                either in or out of isolated block but at least one side is inside and at least one outside.')
            
            newMainLink = (oldToNewInd[mainLink[0]-1][0]+1,oldToNewInd[mainLink[1]-1][0]+1)
            newPairedLink = (oldToNewInd[pairedLink[0]-1][0]+1,oldToNewInd[pairedLink[1]-1][0]+1)
            newSubLink = (oldToNewInd[subLink[0]-1][0]+1,oldToNewInd[subLink[1]-1][0]+1)
            newPairedLinks.setdefault(newMainLink,{})[newPairedLink] = newSubLink
    
    newInterconnectedLinks = {}
    for linkC,linkList in interconnectedLinks.items():
        if (linkC[0]-1 not in oldToNewInd and linkC[1]-1 not in oldToNewInd):
            continue
        elif (linkC[0]-1 not in oldToNewInd or linkC[1]-1 not in oldToNewInd):
            raise RuntimeError(f'One side of link {link} (key of interconnectedLinks) is found in the isolated block, and another one not. \
                It is against definition of isolated block.')
        newLinkC = (oldToNewInd[linkC[0]-1][0]+1,oldToNewInd[linkC[1]-1][0]+1)
        for link in linkList:
            if (link[0]-1 not in oldToNewInd and link[1]-1 not in oldToNewInd):
                continue
            elif (link[0]-1 not in oldToNewInd or link[1]-1 not in oldToNewInd):
                raise RuntimeError(f'One side of link {link} (link from values of interconnectedLinks for key {linkC}) is found in the isolated block, and another one not. \
                It is against definition of isolated block.')
            newInterconnectedLinks.setdefault(newLinkC,set()).add((oldToNewInd[link[0]-1][0]+1,oldToNewInd[link[1]-1][0]+1))
        if len(newInterconnectedLinks[newLinkC])==0:
            # Is it correct procedure?
            newInterconnectedLinks.pop(newLinkC)
        
    # Update blockEdges
    newBlockEdges = {}
    for comp,blockLength in blockEdges.items():
        if comp-1 in oldToNewInd:
            newBlockEdges[oldToNewInd[comp-1][0]+1] = blockLength
    
    return newFromComponentLinks,newToComponentLinks,accStarts,accEnds,newLinkLengths,newPairedLinks,newInterconnectedLinks,newBlockEdges

In [ ]:
#| export
def removeIsolatedBlocks(isolatedBlockList,components,componentLengths,
                         fromComponentLinks,toComponentLinks,
                         accStarts,accEnds,linkLengths,pairedLinks,interconnectedLinks,blockEdges):
    if len(isolatedBlockList)>0:
    
        newToOldInd = list(range(len(components)))

        for isolatedBlockStart,isolatedBlockEnd in sorted(isolatedBlockList,key=lambda el: el[1],reverse=True):
            # deleting components and creating translation from new index to old index
            for compToRemove in range(isolatedBlockEnd,isolatedBlockStart-1,-1):
                del components[compToRemove-1]
                del componentLengths[compToRemove-1]
                del newToOldInd[compToRemove-1]

        # Converting new to old index to old to new index
        oldToNewInd = {el:[ind] for ind,el in enumerate(newToOldInd)}
        # Converting new to old index from list to dict (not sure it is needed in the first place)
        # newToOldInd = {ind:[el] for ind,el in enumerate(newToOldInd)}

        # `oldToNewInd` as well as `newToOldInd` are 0-based both index and values. To connect them to links (which are 1-based), 
        # for indexing you need to do linkComp-1, to get back to links, +1, `oldToNewInd[linkComp-1][0]+1`

        # updating links
        fromComponentLinks,toComponentLinks,accStarts,accEnds,linkLengths,pairedLinks,interconnectedLinks,blockEdges = \
            updateLinksRemoveComp(oldToNewInd,fromComponentLinks,toComponentLinks,
                                  linkLengths,pairedLinks,interconnectedLinks,blockEdges,
                                  accStarts,accEnds)
    else:
        oldToNewInd = {}
        newToOldInd = []
    return components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,\
            linkLengths,pairedLinks,interconnectedLinks,blockEdges,\
            oldToNewInd,newToOldInd

### Clearing small element wrapping function

In [ ]:
#| export
def clearInvisible(zoomLevel,linkLengths,pairedLinks,interconnectedLinks,blockEdges,
                   fromComponentLinks,toComponentLinks,
                   accStarts,accEnds,
                   components,componentLengths):
    print('Removing links according to collapsible blocks')
    
    # ❗TODO This process should be changed to linkLengths dict using paired links
    numOfComponents = len(components)
    # Link removal
    processCollapsibleBlocks(zoomLevel,linkLengths,pairedLinks,interconnectedLinks,fromComponentLinks,toComponentLinks)

    blockEdges = clearRearrangementBlocks(zoomLevel,blockEdges)
    
    # Identify empty (without links) edges
    leftEmptyList,rightEmptyList = findEmptyEdges(fromComponentLinks,toComponentLinks,accStarts,accEnds,components)

    # Identify isolated blocks ,i.e. without any links on both sides and no outside links from inside
    isolatedBlockList = identifyIsolatedBlocks(leftEmptyList,rightEmptyList,fromComponentLinks,toComponentLinks,components)
    
    # Remove identified isolated blocks
    components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,\
    linkLengths,pairedLinks,interconnectedLinks,blockEdges,\
    oldToNewInd,newToOldInd = \
    removeIsolatedBlocks(isolatedBlockList,components,componentLengths,
                         fromComponentLinks,toComponentLinks,
                         accStarts,accEnds,
                         linkLengths,pairedLinks,interconnectedLinks,blockEdges)
    
    print(f'All links associated with collapsibleComponents <{zoomLevel} were removed. \
    {numOfComponents-len(components)} components were deleted as isolated.')
    
    return components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,\
            linkLengths,pairedLinks,interconnectedLinks,blockEdges,\
            oldToNewInd,newToOldInd

## Exporting layer

These functions, with the main one being `exportLayer`, are exporting prepared zoom level (cleaned and collapsed by other functions) into
Pantograph Visualisation tool data structures (JSON chunk files).

In [ ]:
#| export
def createZoomLevelDir(outputPath,outputName,zoomLevel):
    '''
    Creates a directory for zoom level chunks. The function will take care of correct directory level separator.
    
    Parameters
    ==========
    `outputPath`: str. Absolute path to location where the visualisation data should be created
    `outputName`: str. The name of the visualisation case, will be a directory at the path given by `outputPath`
    `zoomLevel`: int or str. zoom level (maximum number of nucleotides in a single column/bin).
                 A directory will be ceated
                 
    Return
    ======
    The function does not return anything, but creates a directory tree `outputPath`, 
    within it creates a directory `outputName` and inside it creates a directory `zoomLevel`
    '''
    
    os.makedirs(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}',exist_ok=True)

In [ ]:
#| export
def finaliseChunk(rootStruct,zoomLevel,chunk,nucleotides,nBins,chunkNum,curCompCols,prevTotalCols,outputPath,outputName):
    endChunkBin = chunk['components'][-1]['last_bin']
    endChunkCol = chunk['components'][-1]['lastCol']
    chunk['last_bin'] = endChunkBin
    chunk['last_col'] = endChunkCol
    
    localPath = f'{outputPath}{os.path.sep}{outputName}{os.path.sep}{zoomLevel}{os.path.sep}'
    
    fileName = f'chunk{chunkNum}_zoom{zoomLevel}.schematic.json'
    
    with open(f'{localPath}{fileName}','w') as f:
        json.dump(chunk,f,cls=NpEncoder)
        
    rootStruct['zoom_levels'][zoomLevel]['files'].append({
        'file': fileName,
        'first_bin':chunk['first_bin'],
        'first_col':chunk['first_col'],
        'last_bin':chunk['last_bin'],
        'last_col':chunk['last_col'],
        'compVisCol': chunk['compVisCol'],
        'chunkVisCol': sum(chunk['compVisCol'].values()), # Is it necessary?
    })
    
    if nucleotides!='':
        fastaName = f'seq_chunk{chunkNum}_zoom{zoomLevel}.fa'
        rootStruct['zoom_levels'][zoomLevel]['files'][-1]['fasta'] = fastaName
        
        with open(f'{localPath}{fastaName}','w') as f:
            f.write(f'>first_bin:{chunk["first_bin"]} last_bin:{chunk["last_bin"]}\n')
            f.write(nucleotides)
        
    chunk = deepcopy(chunkTemplate)
    chunk['first_bin'] = endChunkBin + 1
    chunk['first_col'] = endChunkCol + 1
    return rootStruct,chunk,0,chunkNum + 1,prevTotalCols+curCompCols,0,'' #rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols

In [ ]:
#| export
def addLinksToComp(compNum,components,fromComponentLinks,toComponentLinks):
    component = deepcopy(components[compNum])
    numLinks = 0
    
    toCompDict = toComponentLinks.get(compNum+1,{}) # !!! Check that in fromComponentLinks component numbering is 1-based
    if '+' in toCompDict:
        for fromCompNum,fromCompDict in toCompDict['+'].items():
            fromComp = components[fromCompNum-1]
            for fromStrand,participants in fromCompDict.items():
                doAddArrival = True
                if fromStrand=='+':
                    upstreamBin = fromComp['last_bin']
                    upstreamCol = fromComp['lastCol']
                    fromRight = True
                    if component['first_bin']-upstreamBin==1:
                        doAddArrival = False
                elif fromStrand=='-':
                    upstreamBin = fromComp['first_bin']
                    upstreamCol = fromComp['firstCol']
                    fromRight = False
                else:
                    raise ValueError(f'Unrecognised direction of node link {fromStrand}')
                if doAddArrival:
                    if upstreamBin + 1 != component['first_bin']:
                        numLinks += 1
                    component['larrivals'].append({
                            'upstream': upstreamBin,
                            'downstream': component['first_bin'],
                            'upstreamCol': upstreamCol,
                            'downstreamCol': component['firstCol'],
                            'otherSideRight': fromRight,
                            'participants': participants
                        })
    
    if '-' in toCompDict:
        for fromCompNum,fromCompDict in toCompDict['-'].items():
            fromComp = components[fromCompNum-1]
            for fromStrand,participants in fromCompDict.items():
                if fromStrand=='+':
                    upstreamBin = fromComp['last_bin']
                    upstreamCol = fromComp['lastCol']
                    fromRight = True
                elif fromStrand=='-':
                    upstreamBin = fromComp['first_bin']
                    upstreamCol = fromComp['firstCol']
                    fromRight = False
                else:
                    raise ValueError(f'Unrecognised direction of node link {fromStrand}')
                if upstreamBin - 1 != component['last_bin']:
                    numLinks += 1
                component['rarrivals'].append({
                        'upstream': upstreamBin,
                        'downstream': component['last_bin'],
                        'upstreamCol': upstreamCol,
                        'downstreamCol': component['lastCol'],
                        'otherSideRight': fromRight,
                        'participants': participants
                    })
    
    fromCompDict = fromComponentLinks.get(compNum+1,{}) # !!! Check that in fromComponentLinks component numbering is 1-based
    if '+' in fromCompDict:
        for toCompNum,toCompDict in fromCompDict['+'].items():
            toComp = components[toCompNum-1]
            for toStrand,participants in toCompDict.items():
                if toStrand=='+':
                    downstreamBin = toComp['first_bin']
                    downstreamCol = toComp['firstCol']
                    toRight = False
                elif toStrand=='-':
                    downstreamBin = toComp['last_bin']
                    downstreamCol = toComp['lastCol']
                    toRight = True
                else:
                    raise ValueError(f'Unrecognised direction of node link {toStrand}')
                if component['last_bin'] + 1 != downstreamBin: 
                    numLinks += 1
                component['rdepartures'].append({
                        'upstream': component['last_bin'],
                        'downstream': downstreamBin,
                        'upstreamCol': component['lastCol'],
                        'downstreamCol': downstreamCol,
                        'otherSideRight': toRight,
                        'participants': participants
                    })
    if '-' in fromCompDict:
        for toCompNum,toCompDict in fromCompDict['-'].items():
            toComp = components[toCompNum-1]
            for toStrand,participants in toCompDict.items():
                doAddDeparture = True
                if toStrand=='+':
                    downstreamBin = toComp['first_bin']
                    downstreamCol = toComp['firstCol']
                    toRight = False
                elif toStrand=='-':
                    downstreamBin = toComp['last_bin']
                    downstreamCol = toComp['lastCol']
                    toRight = True
                    if component['first_bin']-downstreamBin==1:
                        doAddDeparture = False
                else:
                    raise ValueError(f'Unrecognised direction of node link {toStrand}')
                if doAddDeparture:
                    if component['first_bin'] -1 != downstreamBin:
                        numLinks += 1
                    component['ldepartures'].append({
                            'upstream': component['first_bin'],
                            'downstream': downstreamBin,
                            'upstreamCol': component['firstCol'],
                            'downstreamCol': downstreamCol,
                            'otherSideRight': toRight,
                            'participants': participants
                        })
        
    return component, numLinks

In [ ]:
#| export
def checkLinks(leftComp,rightComp):
    leftRdepCond = np.all([link['upstream']+1==link['downstream'] for link in leftComp['rdepartures']])
    leftRarrCond = np.all([link['upstream']-1==link['downstream'] for link in leftComp['rarrivals']])

    rightRdepCond = np.all([link['upstream']-1==link['downstream'] for link in rightComp['ldepartures']])
    rightRarrCond = np.all([link['upstream']+1==link['downstream'] for link in rightComp['larrivals']])
    
    return leftRdepCond and leftRarrCond and rightRarrCond and rightRdepCond

In [ ]:
#| export
def searchIndicesPosRecord(redisConn,redisCaseID,zoomLevel,accessions,posMapping):
    for pathID,posMappingPath in posMapping.items():
        iset_add(redisConn, f'{redisCaseID}.{zoomLevel}.{accessions[pathID]}.Pos',posMappingPath)

In [ ]:
#| export
def exportLayer(zoomLevel,components,componentNucleotides,
                fromComponentLinks,toComponentLinks,
                rootStruct,
                outputPath,outputName,
                maxLengthComponent,maxLengthChunk,
                inversionThreshold=0.5,
                redisConn=None,
                redisCaseID=None,
                accessions=None,
                debug=False):
    
    #Create the directory to hold zoomLevel chunks and fasta files (if available)
    createZoomLevelDir(outputPath,outputName,zoomLevel)

    chunkList = rootStruct["zoom_levels"].setdefault(zoomLevel,{
        "last_bin": components[-1]["last_bin"],
        "last_col": components[-1]["lastCol"],
        "files":[]
    })

    chunk = deepcopy(chunkTemplate)
    chunkNum = 0
    prevTotalCols = 0
    curCompCols = 0
    nucleotides = ''
    nBins = 0
    numComps = len(components)
    numCompsDigits = int(np.ceil(np.log10(numComps)))
    
    if redisConn:
        accessions = rootStruct["pathNames"]
    posMapping = {}
    geneMapping = {}
    for compNum in range(numComps):
        if debug:    
            print(f'Recording component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}')
        else:
            print(f'\rRecording component {compNum+1:0{numCompsDigits}}/{numComps:0{numCompsDigits}}',end='')
        component,numLinks = addLinksToComp(compNum,components,fromComponentLinks,toComponentLinks)
        if redisConn is not None:
            
            for pathID,inv,matrixPathArray in component['matrix']:
                pathRedisStartTime = time.time()
                for binNum,binMatrix in zip(*matrixPathArray):
                    if binMatrix[1]>inversionThreshold:
                        overallBin = component['last_bin']-binNum
                    else:
                        overallBin = component['first_bin']+binNum
                    
                    posMapping.setdefault(pathID,{}).update({int(overallBin):[(int(posStart),int(posEnd)) for posStart,posEnd in binMatrix[2]]})

        compNBins = component["last_bin"]-component["first_bin"]+1
        nBins += compNBins

        if len(componentNucleotides)>0:
            nucleotides += componentNucleotides[compNum]
        chunk["components"].append(component)
        chunk['compVisCol'][component['first_bin']] = compNBins + numLinks
        
        # End of chunk
        if compNum<len(components)-1:
            if len(chunk['components'])>=maxLengthChunk:
                rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                          rootStruct,
                                                                                          zoomLevel,
                                                                                          chunk,
                                                                                          nucleotides,
                                                                                          nBins,
                                                                                          chunkNum,
                                                                                          curCompCols,
                                                                                          prevTotalCols,
                                                                                          outputPath,
                                                                                          outputName)
        else:
            rootStruct,chunk,nBins,chunkNum,prevTotalCols,curCompCols,nucleotides = finaliseChunk(
                                                                                      rootStruct,
                                                                                      zoomLevel,
                                                                                      chunk,
                                                                                      nucleotides,
                                                                                      nBins,
                                                                                      chunkNum,
                                                                                      curCompCols,
                                                                                      prevTotalCols,
                                                                                      outputPath,
                                                                                      outputName)
    if redisConn is not None:
        searchIndicesPosRecord(redisConn,redisCaseID,zoomLevel,accessions,posMapping)
        try:
            redisConn.bgsave()
        except ResponseError:
            pass
    if not debug:
        print()
    
    print(f'Recording zoom level {zoomLevel} finished.')

    return rootStruct

## Main exporter wrapper with its helper functions

This is the main orchestrating function that export a single graph to Pantograph Visualisation tool with a couple of auxiliary functions.

In [ ]:
#| export
def compLinksToAccCompLinks(compLinks,doCompDir=False):
    accCompLinks = {}
    if doCompDir:
        accCompDir = {}
    
    for comp1,comp1Dict in compLinks.items():
        for strand1,strand1Dict in comp1Dict.items():
            for comp2,comp2Dict in strand1Dict.items():
                for strand2,accList in comp2Dict.items():
                    for acc in accList:
                        # Do we need to include strand or component only is enough. Here are two options, just choose one:
#                         accCompLinks.setdefault(acc,{}).setdefault((comp1,strand1),set()).add((comp2,strand2))
                        accCompLinks.setdefault(acc,{}).setdefault(comp1,set()).add(comp2)
                        
                        if doCompDir:
                            accCompDir.setdefault(acc,{})[comp1] = strand1
                            accCompDir.setdefault(acc,{})[comp2] = strand2
    if doCompDir:
        return accCompLinks,accCompDir
    else:
        return accCompLinks

In [ ]:
#| export
def recordZoomLevelForDebug(zoomNodeToComponent,
                            zoomComponentToNodes,
                            zoomComponents,
                            nodeToComponent,
                            componentToNodes,
                            components,
                            zoomLevel):
    '''
    A function which records result of segmentation to dictionaries, 
    which holds results for all zoom levels.
    It is currently used only for debugging purposes and in normal operation 
    all zoom level dictionaries are not created and used.
    
    Parameters
    ==========
    
    Parameters are self-explanatory. Those starting from `zoom` are dictionary
    holding all zoom levels, whereas the same parameter names without `zoom`
    in the beginning are data for zoom level given in `zoomLevel` parameter.
    
    Returns
    =======
    
    Returns modified dictionaries with `zoom` in the beginning of the names. Theoretically,
    it should work without return at all (through pointers passed to the function), but it
    did not work before and to be on the safe side, this is done through return.
    
    '''
    zoomNodeToComponent[zoomLevel] = nodeToComponent
    zoomComponentToNodes[zoomLevel] = componentToNodes
    zoomComponents[zoomLevel] = components
    
    return  zoomNodeToComponent, \
            zoomComponentToNodes, \
            zoomComponents

In [ ]:
#| export
def searchIndicesGeneRecord(redisConn,redisCaseID,geneMapping,genPosMapping,altChrGenPosMapping,genPosSearchMapping,pangenPosSearchMapping):
    '''
    Recording prepared metadata structures into Redis DB
    '''
    for pathID,geneMappingPath in geneMapping.items():
        iset_add(redisConn,f'{redisCaseID}.{pathID}.Gene',geneMappingPath)
    for pathID,genPosMappingPath in genPosMapping.items():
        iset_add(redisConn,f'{redisCaseID}.{pathID}.GenPos',genPosMappingPath)
    for pathID,genPosSearchMappingPath in genPosSearchMapping.items():
        iset_add(redisConn,f'{redisCaseID}.{pathID}.GenPosSearch',genPosSearchMappingPath)
    for pathID,pangenPosSearchMappingPath in pangenPosSearchMapping.items():
        iset_add(redisConn,f'{redisCaseID}.{pathID}.PangenPosSearch',pangenPosSearchMappingPath)
    for pathID,altChrGenPosMappingPath in altChrGenPosMapping.items():
        iset_add(redisConn,f'{redisCaseID}.{pathID}.AltChrGenPos',altChrGenPosMappingPath)

In [ ]:
#| export
def exportToPantograph(graph=None, inputPath=None, GenomeGraphParams={}, 
                       outputPath=None, outputName=None, outputSuffix=None, 
                       isSeq=True,
                       nodeLengths=None,
                       redisConn=None,
                       zoomLevels=[1], fillZoomLevels=True, maxLengthComponent=100, maxLengthChunk=20, inversionThreshold=0.5,
                       debug=False, returnDebugData=False):
    '''
    This function is used by `exportProject` function and should not normally be used independently now.
    '''
    
    
    if graph is None:
        if inputPath is not None:
            print('Loading Genome')
            graph = GenomeGraph(gfaPath=inputPath, isGFASeq=isSeq, **GenomeGraphParams)
        else:
            raise ValueError("Either graph or inputpath to GFA file should be provided")

    if outputPath is None and outputName is None:
        if inputPath is not None:
            if outputSuffix is not None:
                outputPath, outputName = pathConvert(inputPath, suffix = outputSuffix)
            else:
                outputPath, outputName = pathConvert(inputPath)

        else:
            raise ValueError("If inputPath is not given, then outputPath and outputName should be provided.")
    else:
        if outputSuffix is not None:
            outputName = outputName + outputSuffix
    print(f'Recording Pantograph data to {outputPath}{os.path.sep}{outputName}')


    rootStruct = deepcopy(rootStructTemplate)
    rootStruct["pathNames"] = graph.accessions

    if returnDebugData:
        # temporary structure for testing, in normal mode each zoomlevel 
        # should be just saved as soon as it is processed and only previous and current zoom level should be preserved.
        zoomComponentLengths = {}
        zoomNodeToComponent = {}
        zoomComponentToNodes = {}
        zoomComponents = {}  
        zoomCompNucleotides = {}
        zoomAccStarts = {}
        zoomAccEnds = {}
        zoomFromComponentLinks = {}
        zoomToComponentLinks = {}
        
        zoomLinkLengths = {}
        zoomPairedLinks = {}
        zoomInterconnectedLinks = {}
        zoomBlockEdges = {}
        
        zoomOldToNewRemoval = {}
        zoomNewToOldRemoval = {}
        zoomLinkLengthsRemoval = {}
        zoomPairedLinksRemoval = {}
        zoomInterconnectedLinksRemoval = {}
        zoomBlockEdgesRemoval = {}
        
        zoomFromComponentLinksRemoval = {}
        zoomToComponentLinksRemoval = {}
        
        
    if nodeLengths is None:
        nodeLengths = calcNodeLengths(graph)

    pathLengths,pathNodeArray,pathNodeLengths,pathDirArray,pathNodeLengthsCum = initialPathAnalysis(graph,nodeLengths)
    
    if zoomLevels[0]>1:
        zoomLevels = [1] + zoomLevels
    
    if len(zoomLevels)>1 or fillZoomLevels:
        linkLengths,pairedLinks,interconnectedLinks,blockEdges,pathNodeInversion = \
                    getRemovableStructures(nodeLengths = nodeLengths, pathLengths = pathLengths, 
                                           pathNodeArray = pathNodeArray, pathDirArray = pathDirArray,forwardLinks = graph.forwardLinks,
                                           inversionThreshold = inversionThreshold)
    else:
        linkLengths = None
        pairedLinks = None
        interconnectedLinks = None
        blockEdges,pathNodeInversion = getBlockEdges(nodeLengths = nodeLengths, pathLengths = pathLengths, 
                                   pathNodeArray = pathNodeArray, pathDirArray = pathDirArray,forwardLinks = graph.forwardLinks,
                                   inversionThreshold = inversionThreshold)
    
    if returnDebugData:
        initialLinkLengths = deepcopy(linkLengths)
        initialPairedLinks = deepcopy(pairedLinks)
        initialInterconnectedLinks = deepcopy(interconnectedLinks)
        initialBlockEdges = deepcopy(blockEdges)
    
    #Process lowest level zoom to create component structure.
    [numNodes, # number of nodes
    numNodesDigits, # 
    nodeToComponent,
    componentToNode,
    componentLengths,
    components,
    componentNucleotides,
    fromLinks,toLinks,
    fromComponentLinks,toComponentLinks,
    accStarts,accEnds,
    invertedStarts,invertedEnds,
    combAnnotation,combGenPos,combAltChrGenPos,combGenPosSearch,combPangenPosSearch] = baseLayerZoom(graph,
                                      outputPath,outputName,
                                      pathNodeArray,pathDirArray,
                                      pathLengths,nodeLengths,
                                      pathNodeLengthsCum,
                                      maxLengthComponent,
                                      blockEdges,
                                      inversionThreshold=inversionThreshold,
                                      isSeq=isSeq)
    print()
    if returnDebugData:
        zoomNodeToComponent,zoomComponentToNodes, zoomComponents = \
        recordZoomLevelForDebug(zoomNodeToComponent,zoomComponentToNodes,zoomComponents,
                                nodeToComponent,componentToNode,components,
                                1)
    
    if redisConn is not None:
        searchIndicesGeneRecord(redisConn,outputName,combAnnotation,combGenPos,combAltChrGenPos, combGenPosSearch, combPangenPosSearch)
    
    fromComponentLinks,toComponentLinks, \
    linkLengths,pairedLinks,interconnectedLinks,blockEdges = \
    nodeToComponentLinks(components,componentToNode,nodeToComponent,
                         fromLinks,toLinks,graph,
                         fromComponentLinks,toComponentLinks,
                         linkLengths,pairedLinks,interconnectedLinks,blockEdges,debug=debug)
    
    if returnDebugData:
        zoomFromComponentLinks[1] = deepcopy(fromComponentLinks)
        zoomToComponentLinks[1] = deepcopy(toComponentLinks)
        zoomLinkLengths[1] = deepcopy(linkLengths)
        zoomPairedLinks[1] = deepcopy(pairedLinks)
        zoomInterconnectedLinks[1] = deepcopy(interconnectedLinks)
    
    rootStruct = deepcopy(rootStructTemplate)
    rootStruct["pathNames"] = graph.accessions
    rootStruct["pangenome_length"] = np.sum(componentLengths)

    exportLayer(1,components,componentNucleotides,
                fromComponentLinks,toComponentLinks,
                rootStruct,
                outputPath,outputName,
                maxLengthComponent,maxLengthChunk,
                inversionThreshold=inversionThreshold,
                redisConn=redisConn,redisCaseID=outputName)
    
    _zoomLevels = deepcopy(zoomLevels)
    
    if fillZoomLevels:
        if len(linkLengths)>0:
            maxLinkLength = max(linkLengths.keys())
        else:
            maxLinkLength = 0
        if len(blockEdges)>0:
            maxRearrangementLength = max(blockEdges.values())
        else:
            maxRearrangementLength = 0
            
        maxBlock = max(maxLinkLength,maxRearrangementLength)

        while _zoomLevels[-1]<=maxBlock:
            _zoomLevels.append(_zoomLevels[-1]*2)
    
    for zoomLevel in _zoomLevels[1:]:
        
        try:
        
            # Collapsing is done through removing some particular links (not satisfying specific conditions)
            components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,\
            linkLengths,pairedLinks,interconnectedLinks,blockEdges,\
            oldToNewInd,newToOldInd = \
            clearInvisible(zoomLevel,linkLengths,pairedLinks,interconnectedLinks,blockEdges,
                           fromComponentLinks,toComponentLinks,
                           accStarts,accEnds,components,componentLengths)

            if returnDebugData:
                zoomOldToNewRemoval[zoomLevel] = deepcopy(oldToNewInd)
                zoomNewToOldRemoval[zoomLevel] = deepcopy(newToOldInd)
                zoomLinkLengthsRemoval[zoomLevel] = deepcopy(linkLengths)
                zoomPairedLinksRemoval[zoomLevel] = deepcopy(pairedLinks)
                zoomInterconnectedLinksRemoval[zoomLevel] = deepcopy(interconnectedLinks)
                zoomBlockEdgesRemoval[zoomLevel] = deepcopy(blockEdges)
                zoomFromComponentLinksRemoval[zoomLevel] = deepcopy(fromComponentLinks)
                zoomToComponentLinksRemoval[zoomLevel] = deepcopy(toComponentLinks)

            # Process next zoom level
            components,componentLengths,fromComponentLinks,toComponentLinks,accStarts,accEnds,\
            linkLengths,pairedLinks,interconnectedLinks,blockEdges,oldToNewInd,newToOldInd= \
            nextLayerZoom(zoomLevel,
                          components,componentLengths,
                          fromComponentLinks,toComponentLinks,graph,
                          accStarts,accEnds,
                          maxLengthComponent,linkLengths,pairedLinks,interconnectedLinks,blockEdges,
                          inversionThreshold=inversionThreshold)

            if returnDebugData:
                zoomNodeToComponent, zoomComponentToNodes, zoomComponents = \
                recordZoomLevelForDebug(zoomNodeToComponent,zoomComponentToNodes,zoomComponents,
                                        oldToNewInd,newToOldInd,components,
                                        zoomLevel)

                zoomFromComponentLinks[zoomLevel] = deepcopy(fromComponentLinks)
                zoomToComponentLinks[zoomLevel] = deepcopy(toComponentLinks)
                zoomLinkLengths[zoomLevel] = deepcopy(linkLengths)
                zoomPairedLinks[zoomLevel] = deepcopy(pairedLinks)
                zoomInterconnectedLinks[zoomLevel] = deepcopy(interconnectedLinks)


            # Zoom level should be split into chunks and recorded.
            exportLayer(zoomLevel,components,[],
                    fromComponentLinks,toComponentLinks,
                    rootStruct,
                    outputPath,outputName,
                    maxLengthComponent,maxLengthChunk,
                    inversionThreshold=inversionThreshold,
                    redisConn=redisConn,redisCaseID=outputName)
        except RuntimeError as error:
            print(error)
            print(traceback.format_exc())
            break
    
    with open(f'{outputPath}{os.path.sep}{outputName}{os.path.sep}bin2file.json','w') as f:
        json.dump(rootStruct,f,cls=NpEncoder)
    
    if returnDebugData:
        return initialLinkLengths, initialPairedLinks, initialInterconnectedLinks, initialBlockEdges, \
                zoomNodeToComponent,zoomComponentToNodes,zoomComponents,\
                zoomFromComponentLinks, zoomToComponentLinks, zoomLinkLengths, zoomPairedLinks, zoomInterconnectedLinks, \
                zoomOldToNewRemoval, zoomNewToOldRemoval, \
                zoomLinkLengthsRemoval, zoomPairedLinksRemoval, zoomInterconnectedLinksRemoval, zoomBlockEdgesRemoval, \
                zoomFromComponentLinksRemoval, zoomToComponentLinksRemoval, \
                graph

## Project generation

In [ ]:
#| export
def exportProject(projectID, projectName, caseDict, pathToIndex, pathToGraphs,
                  redisHost=None, redisPort = 6379, redisDB = 0,
                  suffix = '',
                  maxLengthComponent = 100, maxLengthChunk = 6,
                  inversionThreshold = 0.5,
                  isSeq = True,
                  zoomLevels = [1], fillZoomLevel = True):
    '''
    This is the only function that should normally be used to export a set of graphs (e.g. a graph per chromosome)
    to Pantograph Visualisation tool as a project (or interconnected structure).
    
    Exporting of each graph creates a case directory <projectID>_<case name><suffix> with `bin2file.json` file which describes the case overall 
    and each zoom level. At the same time, each zoom level is contained in multiple chunk JSON files, each zoom level `n` is in the directory `n` 
    inside the case directory. Each JSON chunk files contains all required information to visualise up to `maxLengthChunk` components at a given
    zoom level.
    
    ALl case directories are in project directory together with `<projectID>_project.json`, which is simply provides association between case names 
    and and corresponding directory name.
    
    Finally, information about the project will be recorded to Pantograph Visualisation tool data index to make it discoverable by the tool.
    
    In addition, no metadata is recorded into these files as it inflates it very quickly. Instead, a very simple (optional) API works alongside main
    Pantograph Visualisation tool which provides a lot of various metadata on request if API available or do nothing if not. This API uses Redis DB
    with special DB schema.
    
    When graphs are exported some metadata (annotations, genome and pangenome positions) can be recorded to Redis DB. If Redis DB is not available or 
    recording of metadata is not needed, then parameter `redisHost` should be omitted. Otherwise, if Redis DB is available and metadata should be recorded, 
    then `redisHost` should be set to the hostname (or IP address) of the Redis DB server
    
    Parameters
    ==========
    
    `projectID`: str - this is a project id and will be the name of directory that holds all data to visualise the project.
                  It is recommended not to have special symbols and spaces in this name
        

    `projectName`:str - Human readable short name describing the project. It should not be more than 50 symbols.
    
    `caseDict`: dict{str:str} - a dictionary of each individual graph within project. Usually one file per chromosome, although, in some cases 
                there can be only one graph for all chromosomes or other cases.
                Graph names can contain path relative to `pathToGraphs` if they are in different directories.
    

    `pathToIndex`: str - along with project data structure, exporting adds information to the index.json file which is
                    read by Pantograph visualisation component in the first instance. If it does not exist at the given path
                    it will be created at the given path.
    
    `pathToGraphs`: str - path where graph files mentioned in `caseDict` can be found.
    
    
    `redisHost`: str or None (Default is 'redis') - A host name of Redis DB instance. Default value is done as it is working with 
                 Docker Compose infrastructure.
    `redisPort`: int (Default is 6379) - Port of Redis DB instance. If `redisHost` is None, then this parameter is ignorred.
    `redisDB`: int (Default is 0) - A DB number for the given Redis DB instance. If `redisHost` is None, then this parameter is ignorred.
    
    `suffix`: str (Default is '') - Suffix is used for the naming of the output directories for each graph (molecule or chromosome), 
              e.g. if project is called 'pangenome_project', case is 'chr1' and suffix is '_new', then the output directory for this 
              graph visualisation data structure will be called 'pangenome_project_chr1_new'. If not set, then no suffix will be added.

    `maxLengthComponent`: int (Default is 100) - defined how large (in visual columns) can the component be. If continuity allows for 
                          longer components, it will be broken into components of maximum `maxLengthComponent` columns. 
                          If not set, default of 100 is used.

    `maxLengthChunk`: int (Default is 6) - defined how many components will be recorded into a single chunk file in the visualisation data structure. 
                      Smaller chunk files allows to load data into memory more precisely, but if a lot of small components should be loaded, 
                      larger chunks make it quicker. If not set, default of 6 will be used.

    `inversionThreshold`: float (Default is 0.5) - Each cell has calculated inversion ratio (see user manual for details). It will be shown as 
                          inverted (shades of red) if its inversion ratio is (strictly) more than `inversionThreshold` set. If not set, default is 0.5.

    `isSeq`: bool (Default is True) - If the graph is not nucleotide graph (e.g. gene graph generated from paths or annotations), `isSeq` should 
             be set to False. Default is True.

    `zoomLevels`: list[int] (Default is [1]) - Zoom levels that should be generated. The number indicate how many maximum nucleotides (or genes 
                  in case of gene graph) can be displayed as single column in the visualised matrix. Zoom level of 1 is added automatically 
                  if it is not in the list. Each previous zoom level should be a factor or the next one, i.e. next zoom level should be obtained 
                  by multiplying previous one by an integer more than 1. E.g. zoom levels [1,2,4,8] is correct, but zoom levels [1,2,4,6] is 
                  incorrect because 4 is not a factor of 6. Defaults is [1], meaning to generate only lowest zoom level (single nucleotide/gene resolution).

    `fillZoomLevel`: bool (Default is True) - If set to True, then on top of the highest zoom level set in `zoomLevels`, the function will keep 
                     adding more zoom levels by multiplying it by 2 until there is no non-linear links to remove left. Default is True
    
    '''
    
    
    zoomLevels = adjustZoomLevels(zoomLevels)
    
    indexFile = f'{pathToIndex}{os.path.sep}index.json'
    if os.path.exists(indexFile):
        with open(indexFile) as f:
            indexDict = json.load(f)
    else:
        indexDict = {}
    
    projectOutputPath = f'{pathToIndex}{os.path.sep}{projectID}'
    projectDict = {}    
    
    for seqID,gfaFile in caseDict.items():

        coreGFApath = f'{pathToGraphs}{os.path.sep}{gfaFile}'
        
        outputName = f'{projectID}_{seqID}{suffix}'
        
        projectDict[outputName] = seqID
        
        print()
        print(f'Processing case {outputName}')
        if redisHost is not None:
            print(f'Using Redis DB server at {redisHost}:{redisPort} with db number {redisDB}.')
            redisConn = Redis(host = redisHost, port = redisPort, db=redisDB)
        else:
            redisConn = None

        startTime = time.time()
        exportToPantograph(inputPath=coreGFApath, GenomeGraphParams={}, 
                                        outputPath=projectOutputPath, outputName=outputName,
                                        isSeq=isSeq,
                                        redisConn=redisConn,
                                        zoomLevels=zoomLevels, 
                                        fillZoomLevels = fillZoomLevel,
                                        maxLengthComponent=maxLengthComponent, 
                                        maxLengthChunk=maxLengthChunk, 
                                        inversionThreshold=inversionThreshold)
        runTime = time.time() - startTime

        print(f'Exporting gene graph for {seqID} took {runTime} seconds')
    
    redisConn.close()
    
    with open(f'{projectOutputPath}{os.path.sep}{projectID}_project.json','w') as f:
        json.dump(projectDict,f)
    
    indexDict[projectID] = projectName
    with open(indexFile,'w') as f:
        json.dump(indexDict,f)
    